In [1]:
import pandas as pd
import warnings
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, roc_auc_score, balanced_accuracy_score,confusion_matrix
from sklearn import svm
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sn
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pylab as plt
import matplotlib.transforms

In [2]:
label = pd.read_csv("Datasets/ADR_combined.csv")
pca = pd.read_csv("Datasets/pca_dataset_for_5000_features.csv")
pca.drop("Unnamed: 0", axis=1, inplace = True)
label.drop("Unnamed: 0", axis=1, inplace = True)

In [9]:
def dbAdd(scores, df1, df4, lim):
    f1_major=0
    recall_major=0
    precision_major=0
    roc_major=0
    acc_major=0

    for i in range(5):
        f1 = scores['test_f1_macro'][i]
        recall = scores['test_recall_macro'][i]
        precision = scores['test_precision_macro'][i]
        roc = scores['test_roc_auc'][i]
        acc = scores['test_accuracy'][i]
        df2 = pd.DataFrame({'Fold': i+1, 'ADR': [adr], 'PCA': lim, 'F1 Macro': f1, "Recall Macro": recall,
                        "Precision Macro": precision, "Balanced Accuracy": acc, "ROC Macro": roc})
        df1 = df1.append(df2)

        f1_major += f1
        recall_major += recall
        precision_major += precision
        roc_major += roc
        acc_major += acc

    df3 = pd.DataFrame({'ADR': [adr], 'PCA': lim, 'Avg F1': f1_major/5, "Avg Recall": recall_major/5,
                    "Avg Precision": precision_major/5, "Avg Acc" : acc_major/5 ,"Avg ROC": roc_major/5})
    df4 = df4.append(df3)

    print(df1)
    print("\n\n")
    print(df4)
    return df1, df4




def getDF():
    df1 = pd.DataFrame({'Fold': [], 'ADR': [],'PCA': [], 'F1 Macro': [], 'Recall Macro': [],
                        'Precision Macro': [], "Balanced Accuracy": [], 'ROC Macro': []})
    df4 = pd.DataFrame({'ADR': [],'PCA': [], 'Avg F1': [], 'Avg Recall': [],
                        'Avg Precision': [], "Avg Acc": [], 'Avg ROC': []})
    return df1,df4





def doClassification(X,Y, adr, lim):
    import math
    k = int(math.sqrt(len(X))//2)
#     classifiers = [
#                    GaussianNB(), 
#                    DecisionTreeClassifier(max_depth=10, random_state=101, min_samples_leaf=15),
#                    RandomForestClassifier(random_state=101, n_jobs=-1, oob_score=True, n_estimators=k),
#                    SGDClassifier(max_iter=1500),
#                    LogisticRegression(random_state=0)
#                   ]

    
#     classifierNames = ["NB", "DT", "RF", "SGD", "LR"]
    
    classifiers = [
                   svm.SVC()
                  ]

    
    classifierNames = ["SVM"]


    for i in range(len(classifiers)):        
        clf = classifiers[i]
        clfName = classifierNames[i]
        print("==================================", clfName, "==================================")
        
        df1, df4 = getDF()
            
        scores = cross_validate(clf, X, Y, cv=5, scoring=scoring, return_estimator=True)
        
        array = [[0,0],[0,0]]
        for i in range(5):
            estimator = scores['estimator'][i]
            y_pred = estimator.predict(X)
            array += confusion_matrix(Y, y_pred)
        
        array = array // 5
            
        sn.set()
        df_cm = pd.DataFrame(array)
        plt.figure(figsize = (7,6))
        ax = sn.heatmap(df_cm, annot=True)
        plt.title(clfName + ' - ' + adr)
        
        if not os.path.exists('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Plots'):
            os.makedirs('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Plots')
        plt.savefig('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Plots/' + clfName + "_" + adr + '.png')
        plt.show()
        
        
        df1, df4 = dbAdd(scores, df1, df4, lim)
        
        if not os.path.exists('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Metrics'):
            os.makedirs('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Metrics')
            
        if not os.path.exists('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Averages'):
            os.makedirs('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Averages')
            
        df1.to_csv('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Metrics/' + clfName + "_" + adr + '_Full.csv')
        df4.to_csv('Results/' + adr + '/' + "PCA-" + str(lim) + "/" + clfName + '/Averages/' + clfName + "_" + adr + '_Average.csv')

In [ ]:
warnings.filterwarnings('ignore')
lims = [1000]
scoring = ['f1_macro', 'recall_macro', 'precision_macro', 'roc_auc', 'accuracy']
adrs = ['AFIB']

for adr in adrs:
    for lim in lims:
        X = pca.iloc[:, 0:lim]
        Y = label[adr]
        print("==================================", lim, "==================================")
        scores = doClassification(X,Y, adr, lim)

================================== 1000 ==================================
================================== SVM ==================================


In [26]:
x = list(pca.loc[0])[:1000]

In [27]:
print(x)

[-9.7225080737056, 6.88240062515319, -28.136657848379397, -11.9963535513922, 13.783473748150598, -15.887833231234401, 17.7537660415612, 20.7622763926821, -34.1574479123138, -13.358567556798999, 52.4584899499427, 3.11340622992482, -0.44911792824107105, -2.7472312245519, -4.8245661570513, -2.9548906611954804, 24.635767221707898, 7.67389466964082, 2.2029820207277404, 45.2598850886479, 8.19125847934175, -1.38860878930312, -44.0188428054906, -25.681539922957, 6.277824238263969, 12.6146436041098, -23.544635337446, -52.2222694702689, 59.2135013364668, 12.2397280964193, -49.702470110974595, -5.274435058859559, 17.0011723660153, 16.4465112228657, 0.40654996262007603, -44.43512799780439, 57.47053690658521, 18.8578495611372, -1.8912196837650002, -0.45076062336403705, 15.003112099149401, 1.84670553200703, -1.5175003308121597, 3.95816307886212, 28.7304886388771, 2.9207362490509396, 0.540985984620636, 1.65951668421536, 3.61615365455773, 21.664883351213497, 19.0495397863742, -2.7580954885992, 9.90267

In [24]:
y = [-9.7225080737056, 6.88240062515319, -28.136657848379397, -11.9963535513922, 13.783473748150598, -15.887833231234401, 17.7537660415612, 20.7622763926821, -34.1574479123138, -13.358567556798999, 52.4584899499427, 3.11340622992482, -0.44911792824107105, -2.7472312245519, -4.8245661570513, -2.9548906611954804, 24.635767221707898, 7.67389466964082, 2.2029820207277404, 45.2598850886479, 8.19125847934175, -1.38860878930312, -44.0188428054906, -25.681539922957, 6.277824238263969, 12.6146436041098, -23.544635337446, -52.2222694702689, 59.2135013364668, 12.2397280964193, -49.702470110974595, -5.274435058859559, 17.0011723660153, 16.4465112228657, 0.40654996262007603, -44.43512799780439, 57.47053690658521, 18.8578495611372, -1.8912196837650002, -0.45076062336403705, 15.003112099149401, 1.84670553200703, -1.5175003308121597, 3.95816307886212, 28.7304886388771, 2.9207362490509396, 0.540985984620636, 1.65951668421536, 3.61615365455773, 21.664883351213497, 19.0495397863742, -2.7580954885992, 9.90267379692597, 16.0824735552913, -19.4959170995857, -7.28169685155033, 7.4305655174546805, -0.802715439398275, 12.1421508812125, -7.7311635667787995, 7.22648362009145, -1.2810253252139798, -13.836077990601002, -6.40377881474045, 2.62219621545877, 31.0682252941272, 2.28554740316561, 1.1656984193901299, 10.980066209265699, 13.303695380724099, 7.316404980897611, -0.8794320064691791, -9.73215269181866, -1.2428819109155602, -53.16608528850411, 22.014931801312198, 14.030778455751099, 11.8639039826304, -12.711217259264, 8.263929898896249, -3.22406703398239, -5.337626363445179, -3.8949505793029906, 4.845796376309799, -1.61048574506039, -6.054959154409981, 7.096293460854009, 7.03595093159933, -0.770292068162361, 5.761726531149971, -2.18615240596661, 5.900547527936539, -3.7540246767656, 2.9663668644563903, 0.9745909142302529, 0.9355695550426141, -1.0310464154305101, -9.91934481286357, -6.94583065899325, 1.89903092689949, -9.00163863342541, 12.2977502965284, -1.08364517082461, -0.816459397127986, -11.459307493074201, 3.12836839158486, -7.104899169506941, 9.126004524215551, -3.05208761170619, -15.543184163559301, -3.45544606003046, 4.543955519071821, 5.2049839065159205, 11.960529588651099, -11.9798988394859, -5.62014650487885, 12.6133380188613, 11.7730607940687, 14.6828052309538, -11.1529863015088, 6.9572401051292, -4.35627932640504, 6.382534922053191, -11.1543465851724, -2.1869878557576, -9.65328570773016, 2.36363681548952, 4.32071465402675, 8.38984524889046, -5.559156231383179, 5.899507057687369, -0.648680646734495, 2.82328870751951, 0.8765875148149129, -4.936714356534161, -1.67071614307228, 3.90478487719877, 2.3450723003254597, -4.34756377892919, 0.6867151651531259, -8.02096974461174, 12.620185108393901, 3.15328338693482, -8.37704708144148, -1.48654503320519, 6.46693902148515, -6.45760558756698, -20.8123028926472, -3.7418085178404903, 0.804321218360926, 4.96233020538526, 0.352924210401156, -6.9117387329399005, -3.7488057802089103, -4.35632542451074, 3.33069970130966, 2.7746021467045896, 8.70683936141536, 5.29645398156728, -9.93808344311184, -4.71637728139148, 0.45177883439756394, -6.419759111442809, 0.662001469462437, 17.1976601165856, -0.55015818756013, 2.0175707321728398, 2.50543015929416, 0.41392773338030897, 0.18539378070159804, -3.1798166698305694, -4.46811942146469, 5.92892833233681, -20.0619831160786, -14.4231811854859, 14.6515878962213, 16.861539099556502, 3.35261120996609, 2.6173974604584704, 2.49266551540799, 3.68439273821316, 1.00664210800355, 4.1011968038142905, -0.886528127362577, -0.657061867037742, -6.24975162868279, 0.6986197590159561, 1.6155567828290303, -4.69963510139547, -3.4442899782675203, -11.0087591253092, 5.796718371242241, 2.93264305053496, -7.93754492278967, -9.073006672371859, 0.36339294312314896, 2.6281002141050296, -0.204166793347303, -4.32853913648141, -3.3396895366565196, 2.46815740691286, 7.890525782383361, 5.18873597491342, -8.82469085898996, 7.955790662402659, -7.14188135465829, -11.533467716298599, 0.131036912452511, -0.6934426781833579, -2.1093960358890502, 3.3727800976163103, -3.64001655508597, -11.525053964826801, -12.0026658525034, -8.524576578791809, -10.457975373883599, -0.6018714480383779, -2.25506283977212, -10.7841026497627, 11.682157669730401, -3.5147782565761805, -1.7894296602583901, -3.4117927940923196, 2.27258069624397, 5.55587600246498, -11.770686905390502, 0.8042853493206941, -4.38867215145545, 2.04699838689853, -3.94886846407525, -6.0657947656014, 14.392409143356302, -10.0770260305115, 6.42382268054386, 5.286269928383191, -2.06591129156572, -2.91690596654657, -8.999648214516279, -1.97767485445272, -1.33215976586398, -3.1797618527348797, 1.77346882819604, -4.85194319745346, 2.54351984266075, 3.1536143090275, -2.02981712436625, 0.967381937723981, 4.50955419204233, 10.150193049544999, 1.8470689948137302, -0.16799747593918302, 2.0739061017579, 10.2319219141733, 4.96520221680096, 3.82721615023655, -5.07855696883996, 3.1004016612320298, -4.60281613168598, 4.74605068698373, 0.9492760022749158, 1.44303533435552, -4.30473018553208, 1.88292202289783, 1.7843882526641899, 0.9660151245241549, -12.221971811954, 3.9630361261203904, -7.37791096508212, -3.32948913019395, -1.81046994649175, -2.98525755552414, 3.1807390594944502, 10.240698394809199, -1.25329565186767, 2.1543671594769997, 1.8325151006265898, -7.06513114195425, -5.05182891435779, -3.42410748258169, 4.98194988802967, 3.6777603916475297, -2.67290719321942, -2.8054540566532498, 5.425678262810191, -3.9810944279406, -1.7528437188673898, -3.44021558812126, 1.4737568516299901, 1.5856348807029501, -3.06690684632765, -1.83592463330523, 5.193021322161839, 3.2413537564135497, 0.27362478868705803, 10.0531302623273, -2.23445397079514, -2.3829714802925297, 0.101901430536231, 3.28064191803334, 7.48524793802256, -2.8784103129346206, -4.19380725865977, -4.56695103761428, 3.02678864803604, 3.35019785646979, 1.13573925846325, 5.51100321219373, 2.19333249085277, 9.430074221085, -2.8606381165595494, -2.72052559251361, -5.351320450104249, -2.6867942384891896, -1.92418673184262, -0.800509372324083, 3.4503382185299802, 1.8989590689193299, -2.9105546741472996, 0.8706987921992579, 1.2087688162709198, -6.48836519041314, 5.35753722959005, -3.03698374756243, 8.05789530909984, -1.69277591553863, -0.42429266708737395, -6.030261626262139, 2.3923190917887203, 3.0407615036083104, 3.3824954814707797, -0.6053461371450649, -3.0460344517010998, 3.53714621648429, 6.64305918808416, -0.987718479087513, 2.16275605483943, -1.59712535702483, 1.22470541495721, 2.78844510164783, 1.51444628044269, 4.7788174940488, -5.061977700083, 8.75550223573715, -1.54375209445705, -1.1503510110218302, 5.1056761855075, 2.3879941776446403, 3.74798955300187, 4.0947005136748205, 0.8104225579028229, 1.7479620214306102, 0.431329906006867, 1.8472567548233998, 4.71552089851779, 1.5875100680243799, 7.525863715879941, 2.01896671116732, -4.48594031213268, 6.692775398618999, 3.98704329700283, -1.1844795586797, -9.24105755085118, -2.8525251573995, 6.890009226165941, -2.6784876522579597, 0.161127633266727, 3.36290287522661, 4.28711395267235, 4.708698836422981, 2.14119334889615, 3.17489912140364, 2.46176787755253, -4.71751765844226, 3.31555075420447, 7.1026188931353795, -0.450402946369478, -4.47259706011115, 0.0306135593028077, 0.936246659071684, -0.713667142382795, -2.8239655523834903, -8.9749285297324, 0.543960016777316, 0.177092790915001, 1.2324575049293502, -1.40872668521776, 2.78748883803683, 4.04779481887013, -1.79038309973824, -4.44793862528955, -0.19576055140825302, -4.33038700674663, -3.4623323327937103, 2.90102563044918, -1.1201013495690402, 1.78025335894079, 2.61642690636456, -2.7937709705483598, 1.6418653434808002, 5.36630506328027, 0.39492816991443797, -2.9942985229174304, 0.22898732720236498, 3.43323777735879, -0.4974188493393111, -3.73042329066229, -4.67947837559204, -6.69252883467013, 7.80509287264101, 2.89800718451531, 2.21241838444982, -1.9666060208774, 5.993684527714031, -0.455789747698883, -2.33698379755777, 0.5067451535054379, -1.93952572037516, 0.06812299949444, 0.717445518841963, -1.11241612140041, 5.489285316462981, 2.94282452387316, -1.88492375584836, -2.64332721143947, -0.847887354096875, -1.16346334823449, 0.277720837520372, 7.61365975200505, -1.3036763281911998, -3.1997928577662598, -4.70836158965609, 2.27757911791221, 0.783975143529688, -2.71321343380131, -3.94013554622894, -8.05004734390987, -1.03364866609784, -4.780598444102731, 0.273868159860623, -2.16987791269681, -1.4692953693705202, -1.31713454839481, -2.2791826144928797, -3.17645550906702, -6.452311291583579, 4.05880792215903, -1.04191536158394, -1.9876720357856799, 0.663989966641945, -5.873873456895209, -0.830372498611737, 0.12535996075592998, -1.3753408705429098, -3.53761647053906, -1.36670916190772, 2.85734362041452, 8.46773088606516, -3.1577963897209003, 1.8899531539337098, -2.1070741149691297, -3.6359335813826603, 0.485528171532998, 3.0794721097928197, -1.05307669087338, -0.13568852230622802, 3.8580838797422703, -0.37696683436043704, -5.28804486917854, 2.10113977584021, -1.97876860041147, -0.404024303460057, 1.45197086503795, -3.3232100993302898, 1.4621908019439598, 1.39631615063921, -2.35379389005449, 0.900043576471837, -3.4023310439068597, -2.31218456538176, 10.5714607297865, -3.13069109038859, 1.88181117334065, -1.23499521399934, 0.11188491719688902, 1.76707823284132, 2.4607860214565296, 5.50522147424906, 1.0386587567532701, 0.674104033203383, -0.271089665031546, -4.98466425926826, 2.28980178882572, 0.6159457391839339, -1.9281235346620602, 2.12414073627437, -1.43323231729024, 0.0034023350442754, 2.28496599205003, 4.181150997079721, 3.65228104513765, 2.28239693379368, 6.858278507392809, -1.03642010206836, 3.0884926116893796, -0.8937265722786208, 3.95345054942012, 4.30541336648412, 0.8091232135687091, -0.9083051273686509, 2.57945449724, 1.4862939693535302, -0.836149819480288, 1.7215029299225801, 4.32347776681726, 3.1143517585440295, 0.302849714985163, 0.7510383478850329, 2.84601155705487, -4.55993124891373, -1.37597137605924, -4.159336270710839, 2.4332356149344103, 0.0108825053800666, 0.994422285560755, 3.6517398446708103, -7.5890907934334, 0.826927439143788, -1.31934821509366, 3.0035759075153603, 1.25187073549398, 0.39495198813682103, -0.772596022087563, -0.7768498857007858, -4.93656909120441, 2.06203681561218, 1.03147140608979, 7.96573604947292, 1.58301065234781, 2.9260085925597, -0.0160374070279222, 4.75968237959649, 3.50594223724733, 4.10460037968137, -3.37694903333201, 0.515043217233047, 3.1404722016286803, -7.60575314973513, -0.210562949264174, -2.48805811211804, 0.699536764552172, 2.8401325016632004, -5.3083218896801, -1.21189924159833, -1.2764462768611102, -3.55349047852556, -2.41314574101703, 0.501363378413005, 1.3042256227504299, -0.12409005383604302, -0.90192697337287, 8.59137350326684, 0.0174684704992576, -0.9853951357155121, 0.0997775401812858, -1.4644061483946398, -0.122566055579172, -1.04466775464561, -1.25398662184302, -1.86642807465691, 1.42113975658528, -1.1354871448482, -0.0190211606166634, -0.9298798119961991, -0.0738945438684964, 2.01028499074294, 0.648498585883525, 1.35054052273184, 0.8787016457900959, -1.2691338321039098, -1.16619588167677, 0.690487267552481, 2.01672330551375, 0.476204990069511, -0.25768993661253997, 0.29015693245389196, -4.2585578577667, 5.625879541872821, -3.3418931933107, 0.25149381147669797, -0.7779649131119141, 1.4988960486685299, -1.7697841578771898, 0.0390781958964447, -0.990152260963085, -1.5818273534171, 0.9252866095492691, -2.8182257436219103, 4.40624064259386, 2.3718958233899703, -0.423093935826259, 0.37863476406835794, 0.7771115719791301, 5.315051962218, -0.7763233114299392, 1.36914992729141, -1.52076372622137, 2.55094850735953, 2.0236442588192003, 0.13980170730992902, -0.8355569505059008, -1.71407400524649, 0.11933808598616, -2.4132234188610497, 2.41068093046387, 0.8741084617491071, 2.51447652056407, 0.044862468862049996, 1.2205803148123702, -0.24379891178277102, 0.0104775240393411, 0.835545067828577, -1.13014124641951, -1.67617421441329, -0.38058412224311, -1.16050433663925, -1.3041350464070902, 2.7726206575646697, 0.267527078133452, -0.621513981500661, -0.252078157512107, 1.6081500257602501, 0.9708519478845279, 1.3206838364566698, -1.71881789534975, 3.56157353269102, -0.43312224592088394, -0.752468815611505, 0.7758228502483431, -0.854074987616529, 0.5991421725149589, 0.49812720507186803, 0.5948759095298221, -0.467495491231636, 0.6535416792404429, 2.22497828520557, 0.5550183847300141, -0.619279297870148, 1.74187082400363, -2.76695735993831, 1.00681701595039, 0.10205093454732099, 0.606369787591151, -0.0743840830947779, -1.03425004075593, 1.6945991793968, 2.13834848970937, 0.8690731079925551, -0.618055721005474, 2.18508768297555, -4.4723781251304, 0.712504389143313, -1.05516577362308, 1.6499448063846298, -2.00382516032241, -0.599250662935947, -2.05932129566112, -0.11638298080028599, 0.45551852785926, -1.28572519932253, 0.945560105182413, -0.7746112506033191, 0.4999622396216071, 0.46039248660260296, 0.47752459644625894, -0.0068954712908081296, -0.7221301297618579, -0.24668163931989898, 1.06634935550395, 1.8151871575064702, -0.29471738892537896, 0.414956130851089, 0.28259116895439196, 0.544839635462569, -0.521525074079067, -0.5724433305107839, 0.595826402939709, 1.03542780579356, -0.398358761095692, 0.03561305871019999, 0.87064449295056, -0.9858757106002108, 0.6260359695055899, 0.36891129862975797, -0.7019659444741569, 0.21543702265143, 0.18806575066884, -0.21238465713688798, -1.18194667764335, 0.6490856867817789, 0.772616263356218, -0.47580345361827203, 0.623847213363785, 0.40226022021620705, -0.11731939169005999, -0.251502792532725, 0.499739038569875, 0.244357172468405, 0.7320611254418051, -0.27858854273663103, -0.194373221652261, 0.62788302351287, 1.13495465789292, -0.641048766556792, 2.7087179915109103, 0.24072021960106696, -0.185125393451784, 0.5675545875737099, 0.21346172964418697, -0.8746273250474809, -0.00162852505091573, -0.0490011978471251, 0.00633625267348988, 0.0317405736394819, -0.670736175354352, -0.6416898498128311, -0.15196170062151002, 0.0697994756920758, 0.120813000211146, -0.47097207553832793, -0.142031709352618, -0.31090635448428305, 0.10392063075847302, -0.340582673672611, 0.32519380015434196, -0.435264005421045, 0.32149914918511496, -0.0689886987151209, -0.12214921531918099, 0.201908385165565, -0.0128610515619912, 0.419980786495577, 0.13708568070083, 0.105443935520422, -0.0582391567975049, 0.7546592076261971, -0.0286574984881311, -0.275768534643381, 0.294877008786292, 0.21903206415722698, -0.7712045665940129, -0.0325268007600492, -0.0491351656541818, -0.362983720897457, 0.461204262297251, -0.477223339244825, -0.355252358067604, 0.528948723457509, 0.0879513502080581, -0.384048864754571, -0.190727570877936, -0.267947528797372, 0.125233717616107, -0.7208654021899099, -0.0133455999912042, -0.059940224009595, -0.17524589621723, 0.0668702258873929, -0.35156470454966704, 0.10456989958619699, 0.194778814059741, -0.304864574416935, -0.120859823658331, 0.0125168079242907, 0.46506179901411204, 0.10196789396537001, 0.11563548417356599, -0.16251016587335, -0.631305258874546, -0.18225108869639803, 0.162242352378877, -0.131891499044716, -0.0183202603847824, 0.509065760823935, 0.577680858486054, 0.0194207688624492, 0.277034200842448, -0.314671316211386, -0.275856573511573, -0.14968345528165197, 0.7548670756981709, -0.0676493866310261, 0.0600931051175357, 0.490241060679086, 0.302442331357582, 0.0131690062284639, -0.0634633465811707, 0.18152963557789198, -0.45212980391793295, 0.390752449907109, 0.513687833601401, -0.0220405114600084, -0.31763553472736605, -0.174361155925822, -0.4136297236397271, -0.0812211556718076, -0.000536472041202041, -0.018369947900148998, 0.0960372766758528, -0.0149184763107441, 0.297372586450364, -0.277868457282456, -0.35829131612016996, 0.206196336058429, 0.22037611006292898, -0.444213916964819, 0.18939159850125398, -0.195082916168974, 0.149143994671411, -0.0255833375823964, -0.0432145913126888, 0.173769154443327, -0.0442160752799843, 0.0918734909522208, -0.0691252570824161, -0.048062785179104006, 0.0357679485660478, -0.127336332914651, -0.271279873574408, 0.26335226980758203, 0.0323931024275707, -0.0636873989988316, -0.163038314069949, 0.0781134143829611, -0.0482283240958552, 0.0661067154061924, -0.12248984545069, -0.22328915120320897, -0.0267748003589465, -0.10536845893452698, 0.067562707131071, 0.0934844492026831, -0.0722131329101072, -0.315018992863787, -0.0014219882723928, 0.18722382248110297, 0.202204900613494, 0.0185761245302664, 0.0569650464058603, 0.0848514936938799, -0.0441889614293911, 0.135259372265135, 0.0261437305717033, -0.12096538494771199, 0.0835921274204478, -0.144929645190676, -0.0168232550320424, 0.773564776902122, -0.24160990847377398, -0.43589231173856496, 0.0130645284350786, -0.16297600031854, 0.0970356527548256, 0.0308837188305184, -0.0177680006781611, -0.0219185586845276, -0.0351664421594883, 0.00734913080090262, -0.00364305867665225, 0.0873016041345673, -0.0572813127217252, 0.0810332341751321, -0.0391448608727702, 0.0663348325793047, 0.0128508820802112, -0.0279394800008813, 0.0035083782930107, 0.0256952268976697, -0.008452479886290741, -1.11782429106441e-12, 3.9906827055566706e-13, -5.041766628106971e-13, -9.86493808344394e-13, -4.972770143389939e-13, -1.38350300159746e-12, 9.59418265538578e-13, 2.29544112648772e-13, 2.90689620169218e-13, -9.261486630520571e-13, -3.59115002038307e-13, 1.09463234396853e-12, -3.76823211067891e-13, 6.04596024279154e-13, 1.2738177265333301e-12, -2.95559887866208e-13, 1.04676726990314e-12, -6.654644597592111e-13, -7.336483940117841e-13, 2.3773210279925802e-14, -1.21184269645349e-14, -2.7556970461338603e-14, 1.2431687670219098e-12, -3.27074189774055e-13, -5.10889810168021e-13, -1.08700670180866e-12, -1.4022262719093897e-12, 7.32223528675262e-13, -1.26160089215688e-12, 9.006209830744659e-13, -4.27379110872228e-13, 2.6730117974439e-13, -1.68335100204726e-12, -1.00411259781734e-12, -1.09406242364428e-12, 6.241236663887571e-13, 2.01879503315501e-12, -7.21564367724226e-13, -5.592068699854471e-13, 1.95833216506741e-13, 4.3514081182315006e-14, 5.204983750066261e-13, 1.14286405064958e-12, -6.322263367880201e-13, -1.51532473215514e-13, -4.542764391105229e-13, -8.155872391489209e-13, 1.1582793918076598e-12, 1.36177000006709e-12, -8.088686194701301e-13, 8.92505419556694e-14, 6.702126851583771e-13, -2.51356221442386e-13, 5.364370072036339e-13, 3.4417060166841903e-13, -2.33090216219477e-13, 4.97932926275075e-13, 5.210060650239231e-14, 3.2868271595371197e-13, -2.93111883078594e-13, 1.72587980257527e-13, -4.88990286416238e-13, -3.89298852053997e-13, 9.511373931624522e-13, -3.44120407600043e-13, 9.35348587723786e-13, 1.16600763952812e-12, 6.838346928298349e-14, 5.44635074228372e-13, -7.26376239922282e-13, 4.5208716554638894e-13, -1.04027592489373e-12, -2.05244821259755e-13, 7.19023905983757e-13, -8.83136819357644e-13, -7.25879795230131e-13, 3.6890139776989797e-13, -6.972026648415961e-13, 4.1018328376767793e-13, 7.649392699937941e-13, -5.9008161766025e-13, -1.8539755759519896e-13, -2.1835873533541399e-13, -4.77387377237547e-13, 1.5393555608142801e-12, -1.4793653981915697e-13, -1.01120376255555e-12, 2.31872684467129e-13, 9.470097632261792e-13, -1.30660154933299e-12, 7.075243505591579e-13, 1.10747463308496e-13, -2.32818197623476e-13, -9.25698436715288e-13, 5.07584106020357e-13, 1.62576277772232e-13, 6.91164181429557e-13, -3.5645166362059397e-13, -1.54754864576463e-12, 2.18613793499317e-13, 3.46502665159305e-13, 1.72595730472799e-12, 2.5550770902384e-13, 4.07214143397051e-13, -2.6039908321274804e-13, -7.13452520343348e-13, -1.09816182319124e-12, 4.91551976475338e-14, -7.61457138986159e-13, -1.4774042645227602e-13, -3.47780511204264e-13, -9.862500957581579e-14, -4.74058767921818e-15, -1.59942430051211e-13, 5.54504386126564e-13, -4.72916955753616e-13, -9.66002735183336e-13, -4.88720970021562e-13, 3.54000436809019e-13, -4.4939059123642703e-13, -2.59545397853839e-14, -4.43932876991808e-13, -6.037469987385971e-14, -1.52543568147775e-13, 1.0269017132504101e-12, 2.7695491956080604e-13, -4.9660109300653804e-14, 3.4986442080528807e-13, 5.243853081977649e-13, -1.07149552753376e-12, -5.22815221393921e-13, 7.20775611650744e-13, -2.89544608745274e-13, 3.2068648969423103e-13, 4.05437927095469e-14, 9.774277537011811e-14, 9.894152833374749e-14, 1.0728291267256e-12, -7.95804545208148e-13, 2.53104492418678e-14, -1.1429132402667e-13, 8.935797089441899e-13, -1.0533241376733199e-13, 2.9532086021100096e-14, 4.31021790716921e-13, -2.64771309245615e-13, -6.152063935694731e-13, 1.0672676501456401e-12, -2.99745066791764e-13, -3.4432043129562705e-13, 3.2497528872325003e-13, -7.792941313383341e-13, -6.79764258516722e-13, 8.30151716444925e-14, -3.02165952665477e-13, -1.1134602280879199e-12, 1.27464714887955e-12, 1.04733484720912e-12, 1.38048417890292e-13, 1.59983673708569e-13, 1.8543515682129102e-13, 7.54446216877394e-13, -1.20000633302542e-14, -2.0157250439707398e-13, -4.771441357415021e-13, 3.94114866095765e-13, 4.4040628830322904e-13, 2.24415011142285e-13, -5.554696468967328e-13, 1.1469782971563102e-12, -3.3895527743599995e-14, -1.53291880611703e-13, 1.25218864324194e-12, 1.2699209745421002e-13, -6.13500424315153e-14, -8.339692143891909e-13, 4.82734766458242e-13, -2.70464360669911e-13, 6.41598711732724e-13, 1.3581798479788899e-12, -1.5376350118975e-12, 5.844134778252789e-13, -4.46172423024733e-13, 3.72543643952767e-13, -2.1579016059635997e-13, -9.33552940786338e-13, 1.8043077896407198e-13, -8.609599135455759e-13, 8.65367792288396e-13, 9.39994070367119e-15, 1.19206853106382e-12, -5.02547166263863e-13, 2.9716349101634995e-13, -2.5867794786868e-13, -3.3910119618247597e-13, 3.9007448931462397e-13, -1.96839495054257e-14, -5.2274787606959606e-14, -1.16234535954843e-13, -8.34257813976323e-14, 8.164719904087971e-15, -6.55362191226037e-14, -2.17586205124328e-13, -1.36133411411966e-13, 5.112600677155349e-13, 2.10232437295945e-13, 5.42674740897964e-13, -4.49681751349527e-14, -3.69301429834789e-13, -3.24814582250419e-14, -2.0255103141373102e-13, 8.33302752015566e-13, -5.1391740244364e-13, 3.42634971428119e-14, -1.44262294692538e-13, -9.44969359210018e-14, -1.6018928422942902e-13, -1.72066374567034e-13, 7.326977205288721e-14, -7.38553099684471e-14, 7.49349843544546e-14, 3.04688754989544e-13, -1.49629246725349e-12, 1.17286817352396e-13, 8.556283755026909e-13, 4.89153489654992e-14, 4.96152219391136e-13, -4.096437956917821e-13, 6.54335074852094e-13, 1.52584390546699e-15, 4.2509616151325106e-13, -6.63460630764903e-13, -4.74187509388826e-13, -4.32367392059356e-13, 2.24090048779906e-13, -4.74710512899158e-13, 2.57598078369642e-13, 1.37517486523785e-13, -4.1642862536350896e-13, -5.30787838118913e-13, 1.4707560002151099e-13, -1.04785110241603e-12, 8.55605931084288e-13, -1.3605144628456698e-13, -2.6869524643482393e-13, -2.5181006327157897e-13, -5.264127549366061e-13, 4.39086940965394e-13, 1.1512130352420702e-12, 2.28697984784725e-13, 9.83131098252231e-14, -2.7006450367959197e-13, -5.643162543726011e-13, 6.160707876622e-13, -1.9661165776942701e-13, -3.9665840570092796e-13, 8.813794649306541e-13, -5.76543705955308e-13, 4.1144344142925703e-13, 6.996752714547221e-13, 2.5158488196304e-13, -1.41424326343202e-13, 2.6171332543062804e-13, -1.43288298309393e-12, 7.969690539707529e-13, 3.0809066317086103e-13, 2.8382449413172605e-13, 1.69974453949724e-14, -8.10073459302063e-13, 5.262871386099e-13, -1.7619214371378197e-13, -2.44299663258322e-14, 7.045398558116349e-13, 2.32538927517702e-13, 9.271640595095009e-15, 3.63995655002284e-13, 4.88693544492515e-14, 5.143789214591301e-13, -3.0119535767630607e-13, -9.2915091846073e-13, -5.532543437735511e-13, -6.24238191161613e-13, -3.51286945917107e-13, 9.38351863960583e-13, 7.03126592577308e-14, 4.79873314224099e-13, 1.26742290295314e-13, 1.5969992080182097e-13, -2.3044170204342606e-13, -6.64089965871845e-13, 4.47224284910431e-13, 1.49076084667863e-13, -2.4865167714233196e-13, 7.14989452880876e-13, -3.5076370899284504e-14, -6.75274364744168e-14, -4.831104687656261e-13, 1.0805563826457498e-13, -1.04150202026899e-13, 8.59191703549365e-13, 2.8559064955958395e-13, 2.36249360097155e-13, -3.974556637410711e-13, -3.49671214430085e-13, -2.4868253166325697e-14, -6.244154793108579e-13, 7.37839257950491e-13, -1.86726874896458e-13, 4.917163002907859e-13, -2.2297547658972604e-13, -2.9056778982458e-13, 3.7957382408708003e-13, 8.041070158412679e-13, 8.680889505841091e-13, 5.971541120268922e-13, 4.0884259625732e-14, 1.41890324396563e-12, -2.7193685108082597e-13, 3.01146183037998e-13, 7.668625610084649e-13, -4.93378460867551e-13, -4.740308258303551e-13, -9.79095099622232e-13, 1.12393530791222e-12, 1.67290134849629e-13, -5.701808649526249e-13, -9.49575588225261e-14, 2.97990234913236e-13, 3.5228022997923896e-14, 5.23444822417097e-13, 2.7892412104739696e-13, 1.0705658439210302e-13, 2.2234269450960997e-13, 5.77230398249023e-13, -3.7706197699158504e-13, -9.51039045100404e-14, 2.8971466910394803e-13, 6.690279558792591e-13, 1.08354690798206e-13, -2.0205506061623702e-13, -4.0229137036105796e-13, 9.78238753170488e-14, -3.65348653123022e-15, -1.20033222873491e-13, -1.7261424259498499e-13, -2.95164849584697e-13, 3.26998951911305e-13, -2.03800313483657e-13, -2.69636856331126e-13, 1.22475802832521e-13, -2.3529629751304502e-14, -1.16867715076567e-13, -1.5976433366010198e-13, 8.861903270494019e-14, -1.0043300993240899e-13, -5.166544928595069e-13, -3.42187969906906e-13, 3.27699340254445e-13, 3.89227871483644e-13, 2.08876325521951e-13, 1.42409556472198e-13, 3.03459698676365e-13, -3.321983547916961e-13, 4.79307557479927e-13, 3.9806703887720296e-13, 2.58669573982926e-13, -1.34435396991205e-13, -4.5397417521405295e-13, 3.28454523451164e-15, 5.19593630430049e-13, 2.9366433175469105e-13, 1.0639931412656401e-13, 1.9009276100423001e-13, -9.0268782168284e-15, 2.63635940085979e-13, 1.97518974468534e-13, -7.7278815164491e-14, -2.90984506648635e-13, 4.1962260719860796e-13, -1.59167301930069e-13, 1.66551131794922e-13, 3.96605632191294e-13, -1.29684879048125e-13, 8.41585088627999e-13, -2.8418167554435806e-13, -7.941551963549609e-14, 5.35036253214899e-14, 3.94812994021036e-13, -3.6501774247040794e-13, 6.964209221864e-13, -2.49275829752026e-13, 4.71675975146502e-13, 6.896096692494161e-13, -8.437121139650281e-14, -3.62826673950401e-13, -1.7920274084806502e-13, 4.60721680822896e-13, -1.31155844051648e-13, -1.2204147088145e-13, -2.98120601831731e-13, -3.565126234524e-13, 4.5454442519923297e-13, -5.00037596057687e-13, -4.61861458660459e-13, -1.1065634652163802e-13, 6.25076841348772e-14, -7.77831095611894e-14, 2.5499896792878397e-13, -4.43826363762014e-13, 3.34121556102775e-13, 2.8295164035114e-13, 2.7347851724866e-13, -3.60355688172382e-13, 1.2769249826367402e-13, -1.14297730184375e-12, 2.8193649721666604e-13, -6.96971400689283e-14, 6.278570390434691e-14, 3.28701179724458e-13, 2.7913899042888905e-16, 1.04068204100806e-13, -2.11515427902303e-13, 1.4933316987358498e-13, 1.50249480934809e-14, 1.84235528877073e-13, -2.25572396504651e-13, 2.6500587222397105e-13, 4.96673473585293e-13, 3.45269539260439e-13, 2.37796053298669e-13, 2.53891535649026e-13, -2.48910063423019e-13, 2.2141099864173502e-14, -2.58744958529289e-13, 1.5194306720332199e-13, 5.2763511756509e-14, -1.2783041736053502e-13, 4.7544887019026393e-14, 6.137556443444451e-14, 5.11833614446746e-14, -6.57021577563498e-14, -8.55679648507091e-15, -7.94941014458633e-14, 4.42184168693411e-14, -5.27721009726112e-14, 1.47765561025059e-15, -3.2953353704627395e-14, 8.831353152924049e-14, 3.56985809945313e-14, -9.64384514717218e-14, 6.0357446264979e-14, -5.1129936750121094e-15, 8.164975306999699e-15, -2.90526249181256e-14, 7.357649176935079e-14, 6.38417048505156e-14, 5.75989672410627e-14, -6.10290564211655e-14, -1.56210720445088e-14, 2.03198355945886e-14, 5.93154029682791e-14, -2.1912411585133998e-14, 4.00913260768753e-14, 1.2338690812581402e-13, 9.382703764770391e-15, -3.56930937486575e-14, -7.123338039724e-14, 6.41114797016242e-14, -3.93194847148947e-14, -6.662732225599781e-14, -3.40501290047103e-14, -4.7509644821357196e-14, -7.77652291171544e-14, 1.1836369401096402e-13, 1.7202997460410702e-14, 4.63679177982618e-14, 1.2124831595969599e-13, -3.9955997267784794e-14, -1.70548453802991e-14, -2.69884582589185e-15, -4.5913064766247394e-14, 3.70908659790114e-15, 5.42843403409748e-14, 3.85595039379669e-14, 4.7562824462256894e-14, -9.21489264823018e-17, -1.60075943388876e-15, -4.1419573448472905e-14, 4.4843881340989995e-14, -1.5035223858516803e-14, 1.0160722573265998e-14, 5.95745492126724e-14, 3.6968492457384695e-14, -6.6718078727635e-14, -8.662918152882651e-14, 9.65960818567727e-15, -3.26119110122364e-14, -1.1453379105841302e-17, 2.40026062305004e-13, -8.47761445955883e-14, -1.8227674596362605e-15, 2.7481372329811403e-14, 1.15480819071788e-13, -1.0146498732243198e-13, 2.76924976548249e-14, -4.69599563042924e-14, -1.2713022020600503e-13, -1.1858878916699e-13, -6.35685613162228e-16, -7.35281171437444e-14, -1.74721064088839e-13, 1.30830049377039e-13, -2.5718973155045302e-14, 2.93767416123885e-14, -1.00443152768646e-13, -1.13864614518746e-13, 1.0068824980329501e-13, 8.20323210195164e-14, 1.35881808005545e-14, -1.39890668913017e-13, -1.36612093175698e-13, -2.9460965454725597e-15, -5.3044229924553e-14, -1.3960005186358701e-13, 3.18532020579286e-13, 9.942882020627199e-14, 6.876816153496959e-15, -1.0458461698852e-13, -2.3369074200212398e-14, 3.85014499680792e-14, -5.71976547764844e-14, -1.2817296119910303e-13, -9.48738772050256e-14, -1.8314444337408698e-13, 4.71160931283403e-14, -5.80504933946366e-14, 6.44907528373892e-14, 1.40093037171608e-13, -2.19821918141064e-14, 1.2924375563927498e-13, -7.7622620196201e-15, 6.870475677205919e-14, 7.186718380715021e-14, -6.092887012904741e-14, -2.34894464222804e-13, -6.07436511943343e-15, 3.83267044607936e-14, 2.3590947415581603e-14, -7.82369307949339e-14, -4.21017361365417e-14, 8.68317336040982e-14, 9.81019268620558e-14, 2.99471181918966e-13, 1.0894703570099801e-13, 1.41225445836626e-13, 1.48988769514137e-13, 1.2254822143606501e-14, 5.75021038177049e-14, -2.11595005263047e-14, 1.0003337774264901e-14, 1.6198560679744e-13, 1.41912986461009e-13, 2.3064437587334194e-13, -2.5333802714147202e-14, 3.8555252478905895e-13, -8.215821377099839e-14, 2.6356809888786198e-14, -1.2180173410474601e-14, 2.44953567998547e-13, -5.542962818953421e-14, -2.87784991022867e-13, 3.9385531325944103e-13, -6.809965279593071e-14, 1.5559601594479998e-13, 9.04700432028755e-16, 1.2904781066053199e-13, 4.751317880462751e-15, -8.052020873670571e-14, -3.0986650575926897e-13, -1.3128574815463198e-13, -6.59060504259571e-14, -2.5192725459491696e-13, -9.748947695299632e-14, -2.66402150208036e-13, 4.721247113420789e-15, -1.7180924358166298e-13, 2.41753301932657e-13, 1.5453744655505e-13, -4.3970555441685504e-13, -8.417787273390962e-14, -3.50890699093748e-13, 2.86252478780151e-13, 5.81648097707597e-13, 4.98813193994463e-13, 3.7814558767182106e-13, 2.16264483717975e-13, -6.61114010835772e-14, 1.39452761306422e-13, 6.66202497685362e-14, -7.89429814264064e-15, 1.8010484624327498e-12, 2.0660013111702e-13, -3.18058806047215e-13, -6.88373811707455e-13, -1.47620469148349e-13, -3.6002971758121396e-13, -7.27178830627945e-14, 5.49707296949067e-14, 7.562061507059691e-13, -6.911127395347881e-14, 8.40457064544908e-14, -9.504585136830769e-14, 6.653645771252541e-13, 9.735843904691241e-13, -1.9589912128413503e-14, 5.089933413683239e-13, -3.9885709698060206e-13, -2.00044892683388e-13, -1.4607192053439302e-13, 3.4007445265901896e-13, 1.57918905189018e-13, -9.17531633982058e-13, -9.21759950413969e-13, -1.44112962473557e-13, 4.54774588966186e-13, -3.9069362304628295e-13, -6.72644955953149e-13, 4.2086846519607804e-14, 4.75518589047184e-13, -6.836250782194711e-13, 1.29292965745259e-12, 6.562460696394561e-13, 6.78648716129083e-13, -1.6699316467568002e-12, 5.64912834286016e-13, 1.8060306841348298e-13, 1.73503509333476e-12, -4.2970545472860795e-13, -3.1077341103661496e-13, -2.8370191694401394e-13, 1.30425854943577e-12, 3.6607376583023205e-13, -1.7408172984129402e-13, 3.79588126248797e-14, -1.9434967234203896e-12, 1.4276251915339398e-12, 1.25097409383784e-12, -9.1133710716271e-13, -8.425444314250941e-13, 9.480498623911771e-14, 9.26534525406726e-13, -9.66969789066909e-13, -1.03567493758763e-13, 3.9616497914912e-13, -1.87960073373907e-13, -5.69840169716633e-13, -2.0029708131659197e-13, -6.16796754031974e-13, 2.79296694935716e-13, -4.83592833714305e-13, 1.17749740412735e-12, -4.36816170709707e-14, -1.24063711861453e-13, -2.84896208539045e-13, -5.57225781693115e-13, 8.782280346422409e-13, -4.25091446658782e-13, 2.4467995100805503e-13, 4.819381034772779e-13, 4.7299842896191705e-14, 3.5543190127425604e-14, 2.42722728641152e-13, -3.68769123280784e-13, 9.010672792923259e-13, -1.5799156260351598e-13, 2.44579875797239e-12, -4.691373922267761e-13, 6.19897765334937e-13, 4.3608215757690007e-13, 5.801390990788679e-13, -3.6760511778782107e-13, -1.1722513520288497e-13, 3.3023679191664397e-13, 5.73887479990885e-14, -1.8831813698824902e-14, -4.958531057255121e-13, 3.5580399461008294e-13, 1.23158333077257e-13, -1.27627892642733e-13, -1.5287057605560299e-13, 5.12876349865159e-15, 1.8781118552610998e-13, -3.6494334654420895e-14, -4.7494053892855495e-14, 1.9317669865086e-15, 2.3137515281276103e-13, 5.634601355247499e-14, 4.98848428930953e-13, -3.87632927990063e-13, -1.8368125228303303e-13, 8.44956300334644e-14, 8.09281604899469e-14, 6.60811292641517e-14, 3.09340278949025e-13, -4.8210403632918e-13, -1.0766718369749499e-13, 6.6537961244525e-14, -9.1946674757127e-14, -1.1177784200966499e-13, -2.35481254588425e-13, 1.55964355882678e-13, -2.67810090997519e-13, 1.00414712894609e-13, 4.96810112417185e-14, 1.6904228294041e-13, -4.96267574919827e-14, 1.77408391387823e-13, 2.85790115723764e-13, 3.47970749074549e-14, 3.36703997742187e-14, -9.289397725831931e-14, 8.71317371535969e-14, -9.3713672092827e-14, 1.32066566144995e-13, -2.24970914468383e-13, -2.58279441733277e-13, -1.31783942114873e-13, -6.24375303442691e-14, -1.97263516107014e-14, -7.52387474908329e-14, -7.79768767559366e-14, -3.24693319832709e-13, 3.1134761726901896e-13, 1.57870446630431e-13, -7.1658013472649e-14, 1.3282900869243802e-13, -4.614057386276211e-13, 1.73574040793217e-14, -1.1324823191031098e-13, 1.33121877529104e-13, 3.72215490076335e-14, -8.862932078662379e-14, -2.3123267104151395e-13, 1.8166296814017102e-13, -6.88098019528053e-13, 1.1641806094749002e-13, -1.80490028937867e-13, -1.1234893218438e-13, -3.4686331001127197e-13, 1.4747419952201802e-13, -1.67870444716399e-13, -1.54921776688361e-13, 1.9251292189862498e-13, 1.81285823302149e-13, -1.49946701355672e-13, 7.485526585207828e-15, 8.36904489128343e-14, 1.60748453394437e-14, 1.1643484071293901e-13, -2.1262285494347e-15, 1.8425990465291697e-14, -4.69337452764651e-14, -8.276696790817759e-14, 1.51115451751362e-13, 6.278939452923759e-14, 1.4300576694823598e-13, -1.1328026842721099e-13, -2.8024269268404204e-14, 2.5692411139466605e-13, 2.38204829197057e-13, -6.426589971287979e-14, 2.02989900144144e-15, -1.0990609301442998e-13, -5.27205582858261e-14, 3.91931111049396e-13, -4.2587701021381105e-14, 4.6193958334799205e-14, 8.8836470350502e-15, -7.42936670597525e-14, 1.18628171152891e-14, 1.2468891126283502e-13, -1.24410219122482e-13, 3.19740367194385e-13, 1.14108896703072e-13, 6.80271759443116e-14, -2.11272597793664e-14, -3.4957145619660894e-14, 1.12504375340251e-13, -1.57810188774071e-13, -1.9466097605429001e-13, -1.92658973751628e-14, 2.8024616315999196e-14, -1.1144650694367402e-13, -2.778413284296189e-13, 1.08948367510969e-13, 1.0703564728423499e-14, -7.82740511701452e-14, -7.720159940366541e-14, 4.99181448583701e-14, 1.3300296735509698e-13, -1.7964593723868e-13, 1.42466950725115e-14, 1.30549239712471e-13, 2.55763526324784e-13, -1.71259094430174e-14, -1.2753620078224301e-13, -1.68656149232892e-13, -1.68092144526905e-13, 3.50438659497034e-14, -1.9510331609208302e-13, -9.53028683063557e-14, 4.83785952209121e-14, 1.24158470195728e-13, -1.03937203036872e-14, -2.289889326935e-14, 8.20552712612626e-14, -8.98330381880797e-15, -9.82020739307368e-14, -5.05542460954653e-14, 1.56933169857026e-13, 1.42428712360679e-13, 3.0352101375767e-14, 8.829851572028361e-14, 1.14323429159051e-14, -1.63887913587557e-13, 6.56101713255897e-14, 9.89882873327143e-15, -1.8955067228382503e-13, -6.0289736397380406e-15, 1.1666028152659701e-14, -7.03602811239599e-14, -8.520475046668209e-14, 1.0013388909015899e-13, -1.5914519600303303e-13, 1.71245201791567e-13, -6.747000718309459e-15, 1.8308293023263597e-14, -3.5373191187855504e-15, 1.73185513743834e-14, 1.27757796491529e-14, 1.82650725836397e-13, 4.991054495124659e-14, -1.1715070408432e-13, -3.930083735141829e-14, 3.95133280439727e-14, -1.65034772803669e-13, -8.493445194315769e-14, 7.031239408494819e-14, -5.4534323108077e-15, 9.637623410569941e-14, -2.89810047508859e-14, 4.07946068636298e-14, -5.96858709366849e-14, 5.955695686247929e-14, -1.1678024011199e-13, 2.0899207901526798e-14, -1.30728030846822e-13, 9.731888054483009e-15, 1.5724115150836498e-13, -2.0564287703689398e-13, -1.4329585553799998e-14, 1.16037457075176e-13, 4.11921558352236e-15, -3.8492332982900706e-14, 2.34148519134413e-14, -3.6243132373355203e-14, -1.8110174136763199e-16, -5.630393272587111e-14, -2.27277233666508e-14, -1.35168950082206e-14, -4.87433135449827e-14, 4.74982532408089e-14, 4.48596563737096e-14, -1.2040186910922e-13, -5.5260408250654303e-14, 4.5282682244097394e-14, 6.998843609003011e-14, 5.98738025483302e-14, 1.28436858380601e-13, -2.3482558471431e-15, -1.3236680251662498e-13, -1.89204474001888e-14, 7.73206281911378e-14, 8.95066068555705e-15, -1.50004845922346e-14, 1.0054362940197801e-13, 1.1843403971247197e-13, -6.57488088302411e-15, 9.25983871063412e-17, 9.263219380644279e-14, 9.86465229507512e-14, -3.62934302366208e-14, -4.9749055438934093e-14, -1.8585107901984302e-13, 7.06008439180532e-14, -3.36963025642123e-14, -6.45805814075184e-14, 4.8049057100526395e-14, -2.99003661754627e-14, -4.4451384085652604e-14, -1.1817970381383399e-13, -3.44314816173505e-14, -4.67448082371557e-14, 3.1952989831361703e-13, -1.21431276851164e-15, 1.43402165004117e-13, -6.01107041097772e-14, -3.3620618831889496e-14, 3.38056892282176e-14, 7.61504066278155e-14, 8.723808392391309e-15, 6.416805324986879e-15, -2.46334340750923e-14, 6.42026520476556e-14, 2.0518475746324301e-13, -3.1269116196343e-15, 5.92165511571704e-14, 1.6286202500040698e-13, 1.26999446735835e-13, 1.9614203471432997e-14, -1.21392649323348e-13, -1.55349172923661e-14, 4.314216719255719e-14, 1.35675701284741e-13, -1.1457066459353401e-13, -3.1792852195015e-14, -1.2526234117122002e-13, -5.14704070493599e-14, 5.063833311018621e-14, -9.53869944651557e-14, -8.98766745402539e-15, -5.5125992463099807e-14, -8.7332566812655e-14, 5.385012842424341e-14, 1.3271899937811802e-13, 6.60087654272306e-14, -1.6259690713681697e-13, 7.02924960125149e-14, -6.34516679749108e-14, -6.67356640862728e-14, -2.0851359397159198e-13, 2.51041337925627e-16, 4.99105531046712e-14, 3.36349550312609e-15, 1.31793994393659e-13, -8.768404222974029e-14, -3.95021726949135e-14, 1.0253072153459e-13, -7.71127358353613e-14, 3.9852729959877003e-14, 3.53758846687434e-14, 5.35234763213576e-15, 4.02740801571122e-14, 7.22291463444633e-14, 1.4771416587061e-14, -7.03653396700277e-14, -2.47844768148408e-14, -1.51595754137295e-14, 6.30816426898175e-14, -1.76797695248081e-14, 1.9862506429537698e-13, -3.2039699926290003e-14, 1.4699112042869102e-14, -7.64069781433897e-14, 1.32950037318971e-13, 4.79418609807201e-14, 1.22826594131154e-13, 5.4793904339931805e-14, 8.100102180730231e-14, -6.959900524880721e-14, 9.96500922509782e-14, -1.33127072916591e-13, -2.02983920774576e-14, -5.5459621087266895e-14, -9.67231231027109e-14, -6.79830065913107e-14, 1.1240735185563099e-13, 4.1138857460516997e-14, 6.44530949661265e-15, 1.38032071211059e-13, 7.381865988184711e-14, 1.54698919960178e-13, 3.1703902240113095e-14, -1.17731758127405e-14, 8.796957341721589e-16, -1.4654572129841002e-13, 4.4568195889899595e-14, 8.3816963741431e-14, 7.893592804044001e-15, -3.8250312629239093e-14, 9.763406486027301e-14, 5.1381231998179805e-14, 4.00460616074523e-14, -2.1166869898158403e-14, 3.0479966773790696e-14, -1.1862047425425599e-13, 1.02017076521546e-13, -1.8151901687337797e-14, -3.2746759725680098e-15, -5.42725308993363e-14, 7.00790378354695e-14, 1.0355535769506098e-13, 6.15879012966041e-14, 3.66271278319034e-15, -1.4658337144515802e-13, 4.93958188580196e-15, -1.6791777956822e-13, -5.5047841154365596e-14, 5.6786191734730906e-14, 6.73840399879204e-14, -8.356102719505569e-14, 3.03055224446046e-14, -8.926400069180541e-14, -5.541371091803201e-14, -1.7774984550764103e-13, -3.69413717952236e-14, -2.08940403281082e-14, -4.03287999968956e-14, -4.9891278617015994e-14, 6.04153400715513e-14, 1.9574444004282398e-13, 3.95620630442533e-14, -8.25359504747877e-14, 6.50183950771738e-14, -6.674889063401641e-15, 1.168246329292e-14, -2.7590830563503897e-14, 6.2957752109688e-14, -4.7974304527996e-14, 3.21027387824349e-14, -6.50338127247267e-14, 1.0110362524045e-13, 1.4227993943946802e-13, -9.694996465870001e-15, -3.5296982486168405e-14, 9.25123691818882e-14, 9.67245705451892e-14, -1.52343090834285e-13, 6.38509925624979e-14, 1.02017756366134e-13, -6.265359324544249e-14, 9.25962285386501e-15, -1.53551642275473e-13, 1.5828471248556801e-13, -2.4279049948263598e-14, 3.06419075073205e-14, 1.4488484892674803e-14, 2.4944247868958198e-14, -2.1458542391357497e-14, -1.3599861228999001e-13, -1.3306108872079402e-13, 5.06105461334148e-14, -2.5376488712547002e-14, -2.95663718806458e-14, -9.928327774235108e-15, 5.8789141011277105e-15, 2.7348600178358394e-15, -5.86914702003137e-14, -9.02035869466259e-14, -1.09785126814981e-14, -5.6344880061003196e-14, -7.203553003291789e-14, 5.4953557755256704e-14, 6.46672828545845e-14, 2.5227545553565202e-14, -4.99712053774529e-14, 1.43131167347586e-14, -7.57333066433476e-14, -2.85136826092872e-15, 4.5840690960980004e-14, 1.10345458538189e-14, 6.74094474894829e-14, 1.7979925160961496e-14, -2.38947467385452e-14, -8.14723981093049e-14, -3.7654720312295e-14, -4.17105516885323e-14, -5.5046324657993e-14, 5.50931408885989e-14, -6.74261304598854e-14, -6.358610716367209e-14, -8.309498006672971e-14, 4.3606651831801804e-14, -2.24706599914646e-15, -4.47400177474749e-14, -2.40755671332207e-14, -5.2356448254645694e-14, 3.04687714126797e-14, 1.10692734890631e-13, -1.1273808334656e-13, 1.90686658158482e-14, -1.17950786339616e-13, 1.0813828073036998e-14, -3.9128377370396204e-14, -8.28832322511562e-15, -1.3237392443330402e-14, 1.21598618327374e-13, -3.25880942376569e-14, 7.176786256969071e-14, 5.5355093918811706e-15, 1.0041741691786801e-13, 4.77590264798485e-14, 2.46364750904553e-14, 2.03647433877208e-15, -2.77480422884725e-14, -9.69252834042872e-14, 1.3887445142124e-13, 3.9371783876565496e-14, -2.43027479435451e-14, -1.8533107590667105e-15, 3.09242127878364e-14, 8.298850913162599e-15, -7.471993075126821e-14, -1.02832964153776e-14, 3.98435515935804e-14, 4.00172305705371e-14, 5.955336633036679e-14, 7.45895573233138e-14, 3.4932320893891e-14, 1.41944110074396e-14, -2.0743366442131198e-14, -3.78420462845206e-14, -1.34922666801659e-13, -7.36016757696836e-14, 9.41034589164155e-15, -1.9097425943629e-13, 1.90915849193279e-14, -9.90246178631717e-14, 1.1137736569164199e-13, -6.421333524617939e-14, -1.2874584910840799e-14, 2.91136868467075e-14, 3.19710750830734e-14, -4.339809281874289e-14, 6.553118465182451e-14, 1.35361763830138e-14, -1.09052399601146e-13, 4.8654680831758495e-14, 4.64782074860805e-14, 2.23740983104012e-14, 7.41688367326752e-14, 4.92224504027922e-14, 2.5855377251734403e-14, -2.84857582891721e-14, -9.86464336557063e-14, 1.11708631022738e-13, 8.453733045354379e-14, 1.09951624166687e-13, 4.8923340629988294e-14, -4.55368652144021e-14, -8.345333339001659e-14, -4.2270545429958604e-14, 4.4903621494341795e-14, 7.63078253276285e-14, -5.723070224707199e-14, 6.999234564832281e-14, 6.144109034801689e-14, 9.88691882052224e-14, 5.7769478063449894e-15, -6.605512619771811e-14, -8.13040584350486e-14, -1.01605856820585e-13, -4.3359801212636904e-14, 7.6284904177224e-15, 6.31599513306908e-14, 9.89130761517959e-17, 1.8342005640974602e-14, -4.41577711819975e-15, 9.771969034832291e-14, -2.34631178809434e-14, 7.115674777009969e-14, -4.55718680392861e-14, 9.538010099771471e-14, -6.3937096383147e-14, 5.901649704255091e-14, -5.27578260442956e-14, -6.51781886811866e-14, 4.47547045647211e-14, 3.17542057809928e-14, -6.12668005703177e-14, 4.598802793762769e-16, -6.860098258423049e-14, 2.31333532142083e-14, -7.619210762167371e-14, 4.02891752901406e-14, -3.55452007434601e-14, -6.31024529284498e-14, -8.05192279145149e-15, 3.7101222379014604e-15, 1.61121515764976e-13, 3.2186086646342004e-14, 2.64543108343241e-14, -5.90629868766712e-14, 1.32702621428321e-13, 6.879579269580231e-14, -4.94487422305555e-14, 1.8389204322673102e-14, 7.64679216636905e-14, -6.14277154013526e-14, 1.40990999773048e-14, -9.01805068549718e-15, 1.0261930870223e-13, -7.774964625887629e-14, 1.20048796109586e-15, -8.80711131313973e-14, 1.6693868817950102e-13, -1.3577882473869302e-13, -7.98083887403721e-15, -1.2714891410386801e-13, -1.03625144700334e-13, 4.82481157412652e-14, 9.29387289492296e-14, -6.61412571297254e-14, 9.078579626751371e-14, 1.45177006084298e-13, -1.07586899758913e-14, -7.808639850436971e-14, -5.2358913408840496e-14, 1.53551284372402e-14, -3.4171252630163004e-15, -1.8561321371904798e-14, 2.8168713263608798e-14, 2.67580407852243e-14, -1.06095642381474e-14, 3.2389510600060396e-14, 5.11820184879543e-14, -1.06272957309746e-13, -7.77025342496951e-14, -2.18171204493848e-14, 1.78725349102609e-14, -1.3919251678069597e-13, -1.43112040092297e-14, -8.40933683330698e-14, 3.31629457439382e-14, 1.0871342412776301e-14, -7.22148121466237e-14, 7.63802127217448e-15, 1.0167723085461699e-13, -2.94928426585596e-14, -6.55452071944591e-14, -7.59257510148516e-14, 2.2763207198602996e-14, 9.103185036243179e-15, 2.4421591359286402e-14, -5.844810218193481e-14, -3.92403962174734e-14, 1.40640177176365e-14, -4.03124594907778e-14, 1.3514549725429698e-13, 7.744811295709049e-14, -3.221266420812201e-14, 7.326157835301041e-14, -2.41517789862467e-14, -2.72870118829052e-14, 4.56860791469969e-15, -2.23945567982715e-14, 6.376847703928041e-14, 5.5662846998223906e-14, -5.44413780755993e-14, 1.6101720618312596e-14, -1.3447634762525899e-13, 2.0676372874339402e-14, 6.86387552275075e-14, 3.9621833329667e-15, -1.95004883062139e-14, 1.06068171793696e-13, 3.3305250248399395e-14, 4.10054783133854e-14, 5.7387424010969705e-15, 3.32120103159302e-14, -6.93381746428468e-14, -3.63391395973628e-14, 2.3640326817901796e-14, 2.20696279757343e-14, 3.41717928513333e-14, -6.468910725579921e-14, 4.2010235183096e-14, -2.5052100196646002e-14, -6.485701177729861e-15, -4.6294521652127295e-14, -1.26950729191371e-13, 3.5320742756688696e-14, -1.0447138658068302e-13, -7.69755813781439e-14, -8.28160349017243e-14, 1.6145662901464903e-15, -2.7807607813594398e-14, -4.61009901921423e-14, -2.39999399268368e-14, 4.64986828174004e-14, 6.98307764251815e-14, 2.71034659028097e-14, -3.110091759885151e-14, -2.76327630965666e-14, 1.9077954894706697e-14, -1.8136520072788102e-14, 1.98436329468994e-14, 1.27890462694131e-13, 3.09122834845235e-14, -1.0406482540777e-14, -2.1588415484134604e-13, 5.9454098040459e-14, 2.72324869006376e-14, 8.35078499729188e-14, 2.38205232342501e-14, 6.5209774595418495e-15, -1.1743578482471099e-14, 5.70928784304344e-14, -3.5526590238727e-14, -3.02947448792685e-14, -9.580729897105471e-14, 3.62164205114602e-14, 4.59123882088614e-14, 6.768028865519349e-14, 1.01097663004173e-13, -8.76175535169373e-14, -6.78334120197027e-14, -9.184685798201649e-14, -1.27460589795963e-13, 4.17342750472212e-14, 1.26550482171876e-13, 2.16308033473472e-14, 7.856712026289951e-14, 9.759924268258421e-14, 1.11722646790298e-13, 1.12772221485972e-14, 7.79354159051373e-14, -5.6500009832599706e-15, -9.1802520482654e-14, 1.3395163338774197e-13, 8.79728861270083e-14, -4.55066457370599e-14, -1.20344061129086e-13, -6.02827368119931e-14, -1.57230808597084e-14, -2.0024706247803897e-15, -4.683700758947149e-14, -2.31613176975837e-14, 9.15813112940247e-14, -6.12689608493145e-15, -9.56708055097357e-14, 5.194302093943e-15, -2.68973265199271e-14, -1.23255823791162e-13, 6.5120818218203e-14, -5.51979112336208e-14, -2.3565369730661898e-14, -4.0433796965767604e-14, -1.1822970391700802e-13, -2.3011144754232102e-14, 4.38521881673927e-14, 1.03127736555008e-13, -4.39080657206238e-14, 1.99232155493842e-14, 8.58002181495262e-14, 7.94930740771524e-15, 1.32620320069048e-14, -1.9844744406130697e-14, 5.93017725970692e-14, 2.8356143100943398e-14, -6.234639503849359e-14, -3.89691833799623e-15, -1.95102754079728e-14, -3.92331507612454e-14, 5.3693376983000995e-14, 2.3716025626217096e-14, 2.30344222204498e-14, -1.18811681601304e-14, 2.0143302451647e-14, -9.20903516751732e-14, 5.419604463009289e-16, 3.1986433550000497e-14, 7.426190410639799e-14, -5.22366459474137e-14, -1.0263357044796199e-13, -2.32698351062302e-15, -7.86079315112565e-14, 1.8562658431885502e-14, -3.50374005005308e-14, -1.0786262674592899e-13, -4.23779155843925e-14, 9.15321721928542e-14, -1.84187291352314e-14, 9.1279213459502e-14, -2.9095581664646896e-14, 7.8564503152256e-14, 3.13732194163866e-14, 4.77922512389626e-14, 2.96638058907896e-14, -4.778367547067e-14, 7.07463369321936e-14, -1.14086900759181e-14, 6.754858308428609e-15, -6.4212596299615594e-15, -1.8863287351648398e-14, 9.24928447580008e-15, 6.70539882106426e-14, -4.64369069908901e-15, -3.876887474918969e-14, -2.7503456381953302e-15, -3.3146122864312693e-14, 1.1961032756000799e-14, 3.18128768287434e-14, -1.9515947323317904e-14, -1.45552612811478e-14, -1.2481309444976002e-14, -7.287956187738459e-14, 1.8720542862506698e-14, -1.94128507623619e-14, -1.2227419199478099e-14, 9.38782418234252e-15, 5.22002523596267e-14, -2.86174862450993e-14, 8.34081287273678e-14, -8.334226347728579e-14, -7.576426239465241e-14, -5.4878470688164495e-14, -8.564278852094948e-14, 1.2994622194271202e-14, -7.00279266138606e-14, -2.07176652150266e-14, 6.23735600308279e-14, -4.103078786308e-14, -1.2952595120171901e-14, -3.43145341207128e-14, 7.7586603843666e-14, -1.81895846679948e-14, -4.87543449394956e-14, -1.1684007569156598e-13, 3.4165937185412296e-14, 6.25176279564591e-14, -1.76465257484663e-14, -1.4107285765072102e-13, 5.506796707916429e-14, -1.6443275311041198e-14, 7.797620389490829e-14, -3.2081031522660195e-14, -1.97499653051324e-14, 9.57725404794978e-15, -1.3882248643712501e-14, 7.50657722490274e-14, 6.60096728964653e-14, -9.774479960152851e-14, 5.34358823968706e-14, 2.9303083015633302e-15, -5.4235859564925e-15, -5.5436378220220705e-14, 3.7568690741217304e-14, -4.0927723280774303e-14, 3.4341353009326694e-14, -5.86812605585938e-14, 6.32463655787636e-14, 5.138494758296711e-14, -3.83924261320367e-14, -5.4753456656872595e-14, 4.06308201995824e-15, 1.6677368311084197e-15, 1.9641923837995703e-14, 3.3613501880288504e-14, -3.4660221981217997e-14, 6.60786136553162e-15, -1.0106696963404801e-14, 8.959783881309309e-15, 1.26629370203814e-13, -5.93858394845402e-14, 6.68733613649798e-15, 7.90689732353748e-14, 1.0707987046914698e-15, -3.27946125169557e-14, -9.921559898370859e-15, -6.42078070525759e-14, -6.828393769351149e-15, -8.459419953405741e-15, 1.84627906084095e-14, -1.68844757104553e-14, -6.57538972828939e-14, 4.52855198611537e-14, -3.00116926607839e-14, 1.1937643892076599e-13, -5.0231469759285896e-14, 4.38973445537237e-14, 6.647238425319751e-14, -7.92627147873083e-14, -2.28061715796558e-14, -7.15074605701459e-14, 9.971765032956441e-15, 7.25669110173954e-15, 7.199245454156021e-15, 2.5364434050061904e-14, -2.57643085742099e-14, 3.32460203695893e-14, 1.14203149572317e-14, -6.7208373671881505e-15, -1.22242776845471e-14, -7.56555163181852e-15, -6.21521480944835e-14, -5.81630869326372e-14, 7.536427942149219e-14, 8.39616883425868e-15, -4.44874246042161e-14, -3.40179769426321e-14, 1.8855930222790597e-14, 3.21939629204894e-14, 7.653860005165731e-14, -3.62878552703552e-14, -1.1045909176326099e-13, 1.9310345129716e-15, 7.10943091895625e-14, 4.5513679724270306e-14, -4.396340029638911e-15, -2.96767597958652e-14, -4.11765524002892e-14, 6.517866996705281e-14, 1.2583738628074601e-14, 8.55868075479841e-14, -4.0162396393439694e-14, 1.2940965690396903e-13, -4.21879620775451e-14, -2.13804037573795e-14, -4.15040736801106e-14, 6.22339893730731e-14, -9.873441886420579e-15, -1.42580580782904e-14, -7.14219503662246e-15, 2.27066119586357e-14, 1.28832665699556e-14, -5.67834509445805e-14, 1.8288444005632e-14, 4.03697151496239e-15, 8.179487597366851e-14, -9.32009032896189e-14, -2.8031460529914e-14, 6.03291437625335e-14, -2.34258183252361e-14, -3.575064809018899e-15, 2.9306476029611504e-14, 6.764834881838599e-14, -2.76646056449874e-14, 4.65953524133765e-14, 2.21730951340641e-14, -1.1197550802562e-14, 1.75418968975659e-14, 6.3045045670246e-14, -5.0466487936208e-14, -4.71260418563647e-14, -2.1314452603338602e-16, 2.88198916732886e-14, -4.4139141384752295e-14, -2.4166175911204498e-14, -5.5822712920205993e-14, 1.55199322650032e-14, 2.53152158016072e-14, 6.339160081854631e-14, -5.58229810051164e-14, 4.44880729946859e-14, 1.0394376885501199e-13, 1.6831373262196503e-14, 1.07570699290358e-13, 3.5305026796029702e-15, 6.91034267686944e-14, 2.5337635715846e-14, -1.0838675404116699e-14, 1.02321735484669e-13, -1.22675708808549e-13, 3.1071397845549194e-15, 1.60889535999298e-14, -1.9558728617754103e-14, 7.23133181183528e-14, 4.93168296478751e-14, 7.33089516483297e-14, -9.740013332842e-15, -1.38541991048749e-13, 8.80507295596887e-14, 7.85074044321869e-14, 5.7834637296186e-17, 4.08250531631757e-14, 9.108562666289549e-14, -3.84996873127117e-15, 8.304030783853242e-15, -3.33209364506425e-14, 4.8852767306175e-14, 8.36808990835473e-14, 1.0837032214820099e-13, 8.265963485923379e-14, 7.67343686396471e-14, 4.9509593363892195e-14, 6.12708056399522e-15, 5.93524874547111e-14, -2.47530515435615e-15, -5.858986151684109e-14, 5.930415976052429e-14, 2.8768009803604203e-14, 2.54093569844628e-14, 3.39653486016697e-14, 1.18621779590942e-16, -2.36893228274324e-14, -4.89651941506585e-14, 1.33333425877395e-14, 7.899301257510079e-14, -9.053862573487301e-14, -5.79658114161551e-14, -1.0969839806887701e-13, -1.1435393615495e-13, 6.02099719437449e-14, 1.19200298770859e-14, -2.27100280761645e-14, 5.81689571311959e-14, -5.1058850625723704e-14, -2.0264779050689e-15, -1.24934967252588e-13, -7.63494157031553e-14, 2.26364447166009e-14, -1.1310441813182901e-13, 7.85579924516647e-14, 3.18754779161886e-14, -3.09211817540465e-14, 8.288881127617551e-14, 1.7199300705675597e-14, 7.473280677069099e-14, -3.71993754010752e-14, 1.82754152342711e-14, 8.37100624910767e-15, 1.3777528891711902e-13, 2.95180928081841e-14, -5.216363602602951e-14, 7.033331431160181e-14, -7.432658049083299e-14, 5.60851855180112e-14, -1.5004104213955298e-14, 7.12492012436938e-14, 9.030357279595081e-14, -1.1747174588142e-14, 4.14974664189371e-15, -5.9759146398632e-14, 1.22372443860254e-14, 7.68182793040893e-14, 6.54993574920106e-15, -9.71105114830285e-15, 4.46472636187651e-14, 3.7885842564545695e-14, -4.8845995939152997e-14, -2.7773616439084704e-14, -4.8804964318781507e-14, 2.9324083177003605e-14, 1.5233932131435198e-13, 4.68126007902334e-14, 1.5994702698297e-14, -5.20229088829009e-14, -4.373403685754709e-14, -1.63788075084362e-13, -5.513126071162289e-15, 1.06058330691822e-14, -2.4427139995159304e-14, -1.3790195971506798e-13, -1.0936256370589599e-13, 6.439896794110909e-14, -4.0476932546934404e-14, 1.17754751453112e-13, 6.58471174097253e-14, 1.2103852456644698e-13, -1.6688782177318997e-14, 3.73266112664076e-14, -6.44151259928511e-14, -6.40071600721157e-14, -6.8598017826748e-14, 5.3510121989467e-15, -2.2293530359139704e-14, -6.047412856419931e-14, 4.90322395598872e-14, -7.524694542562559e-15, -5.74862434938177e-14, 8.19873274165455e-14, -2.8245727302657797e-14, 1.8172328715555198e-14, 6.76327337309198e-14, -1.16303736181584e-14, -1.20030578392129e-13, -6.87993618214121e-14, -7.619427728719029e-15, -4.6985962280491e-14, -4.1415233144242795e-14, 1.31291946526139e-13, -1.9842937181841803e-14, -3.6619274061859396e-14, 2.62822381640878e-15, 6.52232367093525e-14, 8.902423346255909e-14, 3.9363950908384204e-14, 1.8062870944115798e-14, 5.61189862695875e-14, 3.44376228729615e-14, -5.32924598118488e-14, -2.41126050768238e-14, -8.164864814016901e-14, 5.77211678334779e-14, -3.68648758971065e-14, 8.64205535916963e-15, 1.1429147773827601e-14, -2.3471497230045398e-14, -2.1678924748042197e-14, 1.3545514068704098e-13, -8.489424135169699e-15, 6.07871293338692e-14, 5.19931162696746e-14, 4.98732211037578e-14, -2.56855241479178e-14, -3.61937144775628e-14, -8.824029739504541e-15, -2.23443994262407e-14, 4.97389818787334e-14, 6.6882165367917e-14, -8.85109894645573e-14, -7.15524927378411e-14, 5.05340580139577e-14, 4.9643423350889704e-14, -9.75948557803133e-14, -3.12487226149557e-14, 4.92147375090885e-14, -7.580629635843342e-15, 6.28591675238428e-14, -5.5235232074215005e-15, -2.1862710677170197e-15, 7.950823649941439e-14, 5.6013432821184306e-15, 4.95917449267162e-14, -8.962532954371661e-15, -2.6745837434256e-14, -1.32565326156455e-13, -2.62198635641912e-14, 1.99792893375128e-14, -3.3238595596267e-14, 9.080193305970698e-14, 1.34370353489393e-14, -2.56029823239491e-14, -8.412327793153921e-14, 4.26484570705683e-14, 6.90336084073157e-15, 5.66740443780222e-14, -7.51780514848298e-14, 2.30601455617364e-14, 5.35610731130227e-14, -1.83531600694197e-14, -8.51303205749496e-14, 3.52219339061157e-14, 5.557074797542421e-14, -6.87051549103664e-14, -2.4843527581437898e-14, -5.46175661849814e-14, 5.97137253331379e-14, -2.10585395029776e-14, -5.13291739720593e-14, -2.68658053474373e-14, -1.7590284916045702e-14, -1.0776604633558e-14, 3.4709719010663296e-14, 6.37913558332444e-14, -1.3951948078811702e-14, -2.2353764160821098e-14, -2.10172240747256e-14, -1.63472004568011e-14, -1.2946764713590202e-14, -1.0095506733837e-14, 5.96116084202741e-14, -4.61150212836584e-14, -1.32315412042101e-13, -1.51179924199468e-14, 5.07106661637396e-15, -4.627838993095199e-15, 8.143765965359911e-14, 2.79313301081433e-15, -1.8153989831799e-14, -1.0475578466078698e-13, 7.42950087223628e-15, 5.3908342392890996e-14, -1.06246867504573e-14, 7.892265680059621e-14, -7.93300021227805e-14, -6.714015798855651e-14, 2.2503510552904298e-14, 1.3740308084355399e-14, -3.60561945625694e-14, 1.89867502654337e-14, -4.98857635285438e-15, -8.262664419709141e-15, 4.82384742292241e-14, 7.45254643250309e-14, -4.0882777525493e-14, 4.85061077560931e-15, -4.90236200244418e-15, 5.04078817904906e-14, -9.269774546478841e-14, -3.0618742250302396e-14, 4.08234515825364e-14, 9.48121401079454e-14, -4.2438072366877504e-14, 2.24830832585564e-14, 4.82085387039806e-14, 3.42237650981387e-14, 1.63048873196495e-14, 4.8754235438146205e-15, 2.75867634589893e-14, 6.496643908355009e-14, -6.02130422344681e-15, -9.01356053244014e-14, 6.69574430840347e-14, -1.9928681898954498e-14, 4.2488790891346603e-14, -2.6688284032357605e-14, -3.19865293391261e-14, 4.7686936101357e-14, 1.14190857067962e-13, -2.5140904916104698e-14, 4.16005958639265e-14, 1.1162944884728899e-13, -1.3196019541496199e-13, -5.28336173808166e-14, 1.26832483578544e-13, -4.19599842070598e-14, 3.98293320972018e-14, -6.452996067883691e-14, -1.93268445897085e-14, 3.37382073558978e-14, -4.15927057644038e-14, 3.497746819001439e-14, -7.51531001524025e-15, 3.57706004617026e-14, 1.9267323352766602e-14, -1.70938528743363e-14, 8.91348465197133e-14, 2.6953444068515302e-14, 3.84482220727399e-14, 7.057416959069639e-14, -5.90286759635484e-16, -7.305099238395509e-14, 4.8804662863214904e-15, -3.30106209354247e-14, 1.3365101231894502e-14, 1.5169900944556002e-14, 7.154545646758399e-14, 7.58353747869514e-15, -5.4884288220652605e-14, -4.4030925036288997e-14, -2.24875921389501e-14, 6.56515069768889e-14, 4.83886898257315e-14, 1.50193665882136e-14, -5.4222883357230795e-14, 8.53988888419151e-14, 1.43284824941578e-14, -5.715145569222769e-14, -1.8193160998520802e-14, 1.81973390321609e-14, 8.522872892643191e-14, -3.7123803976873e-14, 5.663788220683709e-14, 1.3541598304505899e-14, -6.74299852382547e-14, -2.7224465226575803e-14, -4.39572025041667e-15, 5.194081664424279e-14, -5.1859857564160496e-14, 4.46599616181703e-14, -1.34565485699463e-14, -4.2789956925368705e-15, -6.01931693880681e-14, -4.3281351251753096e-14, 4.0341331885010007e-14, -3.9887158742063305e-14, 9.64853279250105e-14, 8.46686124405931e-14, -5.923592185078679e-14, 5.927364758831189e-14, -8.082117052910391e-15, 6.22606752613791e-15, 4.7816510799491206e-15, -3.75029088440169e-14, -6.88708882074936e-14, -8.094588474514191e-15, 2.20226549299552e-14, -1.2390763743966e-16, -3.28321574059164e-14, -5.729801308266411e-14, -4.52918892509881e-14, 5.865015403395951e-14, -2.3052796999069e-14, 8.96567540834588e-14, -5.4151226968484005e-14, 7.44214813559617e-15, 6.3386070118376e-14, 6.36341999426019e-15, -6.481674113867961e-14, 2.7020129003136003e-14, -3.45210674625075e-14, -1.0030379175352199e-13, 3.70547264754395e-14, 4.25493221735293e-14, 3.93386143636425e-14, 7.17293730230984e-14, -3.65571744207384e-15, -2.19116264148037e-14, 6.11191389727657e-15, 9.99166620011599e-15, -8.74708913457622e-14, -2.1738284034187e-14, 6.924779496437959e-14, 7.156113838918359e-14, 3.39554799027488e-14, 1.0147393712606698e-13, -1.15170780431638e-13, 1.805510298822e-14, 1.1288405145250499e-14, -1.7972344826731102e-15, 4.489800930078321e-15, -3.1595271943542195e-14, 3.82534861601351e-14, -1.21478658710075e-13, 2.7710730068299898e-15, -8.76735321151262e-14, 4.28407291553404e-14, 4.62034094480585e-14, -1.78540737115295e-14, 1.82132256088259e-14, 3.8035165979967496e-14, -5.4937080241565806e-14, -6.77905345877135e-14, 2.5749701153412798e-14, 7.15475109758714e-14, -2.82135921416561e-14, -1.09503108276061e-13, 5.38019385057995e-14, 3.2774244634777105e-14, 1.63494395290908e-14, -3.93731458326438e-14, 4.57408046845718e-16, -4.68893075959013e-14, 4.28294670501396e-14, 3.40833232824646e-14, -1.92117489545556e-14, 9.724745366925752e-15, 5.93894691802136e-14, -1.23553679471511e-15, -9.21800917317385e-14, -6.772843464390731e-14, 1.61655915271996e-14, -6.15893891944978e-14, 7.133760450635989e-14, 8.267885393187791e-14, 2.27448680597244e-14, -7.39952098662524e-14, 4.95545063307803e-14, 5.419185550326081e-14, 3.57555223260223e-14, 2.9186341463966594e-14, -8.31119768128125e-14, -6.3433366314206194e-15, 1.16076131363389e-14, -2.8395190098455504e-14, 6.663182103662561e-14, -5.1045213141817594e-14, 4.823652025846911e-15, -6.589815112768461e-14, 1.1240097344510001e-14, 2.1672754294912103e-14, 7.56496770454739e-14, 7.56693377068477e-14, 3.71487287015459e-14, 2.01779623221176e-14, 5.966682438903061e-14, 9.38578092376796e-14, -3.2823069287279497e-14, 6.75924861086418e-14, -2.61217012777415e-14, 3.24883844061992e-14, -6.493967519103389e-14, -7.63579972394925e-14, -1.3481508563395202e-14, 2.7061726260974097e-14, -6.25777875181263e-14, -1.93557176959483e-14, -1.9904362859105002e-14, -4.4323482150641003e-14, -1.37046076705741e-14, -1.3923141734271399e-14, -1.11368639544146e-14, -4.4430316653460895e-14, 5.3038230548272305e-14, -8.58251641868759e-14, -1.99445570832482e-15, 2.1609698095713e-14, -2.94649988427982e-14, -3.88189785049223e-14, -6.945512917227099e-14, -2.47089330843718e-16, -1.5440339977967e-14, -3.4404813188676295e-14, -7.65388572827337e-14, 2.2287491406471e-14, 3.00771897777736e-14, 2.6390877049772303e-14, 4.19970321114152e-14, 7.10041969529376e-14, 9.69296469584427e-15, -7.67025576809289e-14, -3.11299797450863e-14, 4.2381324203620796e-14, 1.01769534129502e-14, 3.9474824425387e-14, 8.08813623931788e-14, -5.833703798334819e-14, 1.11910954000119e-14, -1.28836323689388e-14, -6.89743290779037e-14, -2.5613985521880404e-14, 3.04769696584442e-15, 8.01770502189527e-15, -2.16357457022546e-14, -2.9130175649357605e-14, -1.7858819142407298e-14, -5.53929890543637e-14, -4.43628418932427e-14, -3.13661326238959e-14, 7.864309212739328e-15, -6.1091564986755e-14, 8.722868094219971e-14, -1.63962332512963e-13, -4.63385692995995e-14, -1.9259886383061102e-14, 4.3167674188711094e-14, -2.0342028345544997e-14, 1.99804504478448e-14, 4.824567198756659e-14, 3.7637797964585607e-16, 6.9361478535912e-15, 3.614726002896771e-15, -5.20196211040936e-14, -9.13646645126773e-15, -9.22440270697907e-15, -1.66314843773641e-14, -4.154915151594671e-15, -8.65011534573371e-14, -8.266640317695011e-14, -2.09720693951113e-14, 1.0521255310432402e-15, -2.74718218960183e-14, 3.7498902439174296e-14, -8.516316997492758e-16, -3.75743765417036e-14, -4.51282399100826e-14, -9.372931306759519e-14, 1.8657424823025502e-14, -4.17857993490898e-14, 5.4210218080476796e-14, 4.0154868859132697e-14, 7.26300561857858e-14, -8.63649457042961e-14, -8.68297056927962e-16, -2.90021779630315e-14, -3.5652219877109294e-14, 8.618578001188428e-14, -1.43946407875636e-14, -9.53893539656156e-15, -4.27863092156515e-14, -1.3861367844335398e-14, 5.2535683608830405e-14, 4.23639806105952e-15, -7.63035034937132e-14, 5.6710655419398097e-14, 6.21173839592316e-14, 1.4126010838553899e-14, 9.70694812147557e-14, -3.01342582960855e-14, -5.38066508454275e-14, -3.44892697316642e-15, 1.68563581580929e-14, -2.32499799425654e-14, -4.1813598291448804e-14, 3.13883097289042e-15, 9.22175776642493e-14, -1.43553950258172e-14, 5.01597721970371e-14, 3.56099101458764e-14, -8.282366487311141e-15, -5.41086426544049e-14, -4.17811217278957e-15, -6.365259068712641e-14, 1.3832678171396498e-13, -9.875308439692991e-15, -2.8832551317147204e-14, -1.61211264510169e-14, -2.6883616040899198e-14, -3.5305303632845703e-14, 5.732550887588099e-14, -1.74886710120023e-14, -1.5276586873331202e-14, -1.0811525548959802e-14, -6.94169765278175e-14, 4.55263938720353e-14, -7.134965821434241e-14, 1.04486945783013e-14, 4.75386021272499e-14, -1.56814107177608e-14, 5.346719907876641e-14, 7.008277866391759e-16, -1.94585406391887e-14, -4.51791193724788e-14, 8.908030660236391e-15, -7.25616026610078e-15, -9.99631588221681e-15, 4.1830974291103304e-14, -2.44194786733295e-14, 6.73267397564446e-14, 1.15022012012845e-14, 2.8333180103178897e-15, 1.6169978844639e-14, -9.3771973161899e-14, -5.54214160839943e-14, -5.2520798355146494e-14, 4.1047695456039805e-14, -8.31488376628103e-14, 4.3329885813331704e-14, 3.27070500104675e-14, 4.5397107607056397e-14, 1.1092893880003201e-13, 9.01031246130371e-14, -1.3765017218426502e-14, -2.57015352107389e-14, -1.43249100302101e-14, -1.36983172494614e-14, -6.37354639574974e-14, -5.10864954741347e-14, -1.49633790556524e-14, 3.096114242932e-14, 2.00600011239124e-14, 2.89472446480466e-14, -4.1513041161433297e-14, -8.171983385463251e-14, -7.653813609933269e-16, 4.53478584786131e-14, 3.52340260696736e-16, 2.30463606137803e-14, -1.70006686257405e-13, 2.87717971197183e-14, 5.827047664824211e-14, -1.52331641982486e-14, 8.89159120374633e-14, -1.65069618225914e-14, -1.00814022496214e-14, 6.33088442918906e-14, 6.95687091133836e-14, -5.1457407248528794e-14, -6.10114545055607e-14, -1.25743635505916e-14, -4.1840646664402e-14, 4.12531746207723e-14, 3.2686244048568404e-14, -5.9504330600314e-14, -4.47869196698294e-15, 1.63361623205933e-14, 8.93092247250281e-14, -1.61844615349374e-14, -2.6329425225965996e-14, 3.65232878339301e-14, 8.392198670893519e-14, 3.3982533048215e-14, -4.68954099265863e-14, 8.50449013286227e-14, 8.69263941195461e-15, -3.29232124011951e-14, 7.923045383887379e-14, -7.217717882471671e-14, 5.025478998665e-14, 4.04992931706872e-14, 3.29889928120364e-14, -3.4592193385231795e-14, -5.3008499564090895e-15, -9.238032499285801e-15, 7.525659029309301e-14, -3.8744092136732695e-14, 1.67645518608447e-14, 1.04135899006719e-14, 2.45335886148405e-14, 5.337479397067019e-14, 1.93102719671379e-14, 3.57601914669331e-14, 6.81408666379547e-14, 1.02769614226957e-14, -5.21295035099753e-15, 5.44791202031408e-14, -3.52651727930931e-14, 4.113797999297899e-14, -6.362739740263929e-14, -1.26824842095107e-14, -2.41269690892965e-14, 9.341925929453122e-15, 6.17010179797585e-14, 3.5141826326099405e-14, 1.9928185399151598e-14, -6.927477125170061e-14, 4.32212883788148e-14, 1.58496396201735e-14, 8.197126639100679e-15, -5.87125708707032e-14, 1.99483460922269e-14, 4.9239573926586504e-14, 5.625918812768799e-14, 3.126520277659e-14, -4.30392662510887e-14, 8.762625296751049e-14, 4.193328277947229e-14, -2.8339332905887e-14, 6.942928962970469e-14, 1.05577351887477e-14, -3.1446024407871397e-14, 1.28501840473089e-14, 8.284445509201249e-14, 2.1003064380530398e-14, -1.4948338686084198e-14, 3.922423131216e-14, 2.4270100799725298e-14, 1.52616803061124e-15, 7.81864347720871e-14, -1.7352025765230398e-14, 3.37561351404299e-14, 4.89703668921164e-14, -2.0825448762065102e-14, -3.70887593432015e-14, -6.31968498100413e-14, 2.3068159314487e-15, -6.164922610760061e-15, -5.57022174538133e-14, -4.6932226105733903e-14, 5.537618479258319e-14, -5.832287407262099e-14, -1.47892153086528e-14, -6.1183329564425e-14, -1.4886756775568798e-14, 1.93678946747938e-14, 6.59439070323098e-14, 1.65448534822413e-14, 2.02411745415175e-14, -4.18843071272456e-14, 2.04361573809837e-14, 1.48415284256383e-14, -3.9223221109185e-14, -1.6664114610355802e-14, 5.6999813711489e-14, 2.46963289357955e-14, -1.96885398851982e-14, 3.3567033790504096e-14, 3.4270157625264e-14, 4.84331487359243e-14, 2.4750932832437198e-14, -4.3667700439706395e-14, 5.22512825507991e-14, 6.442862155763599e-14, -3.92892785736203e-14, -2.53346645361172e-14, -2.43615437297563e-14, -2.50855122851538e-14, 4.3368112193561205e-14, 1.13920016520279e-13, 5.33669729577672e-14, 3.8367678532129904e-14, 9.961534931871459e-14, -8.384067780667278e-15, 9.13770918346859e-15, -2.0347543868995398e-15, -1.32335645994271e-14, 4.71689116366423e-14, 5.93570866133646e-15, -1.2007697028869301e-14, -4.5741401165570396e-14, -7.38758431793322e-14, 8.18621857800552e-15, -1.2122608141483802e-13, -3.83368731398197e-15, 9.68317637810436e-15, -1.77705635347776e-14, -2.7870984541434002e-14, 4.76343829837802e-14, -2.14515015567662e-14, 5.58202897388977e-14, -9.80954255341339e-14, -7.0137687199735e-14, 6.5869081147676e-15, 3.4741683719419296e-14, -9.58643147324028e-15, -3.16060219854668e-14, 2.30629019036903e-14, 4.097329388632029e-15, 1.48549728582713e-14, -2.56222780306512e-14, -5.75445854339088e-14, -7.755992364712029e-14, -3.91373455174282e-14, -6.72097421455583e-14, 8.753802916685181e-14, -6.66608735059644e-15, -5.77222132057548e-15, -2.00057641680531e-14, -4.65014036364489e-14, 1.24936439862676e-14, 1.9595423515244598e-14, 3.5365606976253306e-14, 6.661228787116029e-15, -6.20086187834574e-14, -2.73308046593609e-14, -6.415856631218351e-15, 8.049318918267159e-14, 7.50154266612265e-14, 7.1384071648583e-14, 4.8860336290650495e-14, 4.0633862999405e-14, 4.9222892449867494e-15, -1.45271111374742e-14, -2.7908312587949798e-14, -4.3118526596997006e-14, 4.03343006751306e-14, -5.96484454040846e-14, 5.8438317824305e-15, -3.74513332496499e-14, 6.6702799265092605e-15, 5.6194316437948896e-14, 5.91476376861961e-14, 3.6260493096273103e-14, 4.43570780336039e-14, 5.0253088570730004e-14, -1.4785928505334603e-14, -2.46785487892852e-14, -3.16951942198437e-14, -5.5218076474853695e-14, -1.7474405498019403e-14, -8.11932915425441e-14, -4.64380775034374e-14, -8.88922316310848e-15, -2.2321329873329897e-14, 7.29459035418208e-14, -8.77234248989023e-14, -4.76052959165753e-15, -6.04223623711952e-15, -2.1714579534573797e-14, -2.64268617166458e-14, 5.1707000178465503e-14, -3.9814719906306604e-14, -1.9623654255430802e-14, -8.108252224087211e-14, -1.08352709433275e-14, -1.0755307958570102e-14, -3.49455987029842e-14, -2.55731607554424e-14, 4.3548498703044293e-14, -9.04864132407631e-15, 1.93899853929592e-14, -7.748655641171889e-16, 1.49910401587529e-14, -1.36832896934394e-14, -4.23626662988024e-14, 5.56166554485525e-14, -3.0656309112326e-14, -5.3039740932489407e-14, -2.38464424613186e-14, 3.65969071683429e-14, -7.40482588786664e-14, 7.766670860581261e-14, 4.5928431893724095e-14, -3.46855101588069e-14, 2.49071169757904e-14, -2.5632722801426e-14, -2.29945741880057e-14, 7.131614029869189e-14, 4.468012781035229e-15, -1.2261513463518e-13, 2.7492782937191897e-14, 3.02042146072079e-16, -8.57975899101933e-14, 7.50791626295201e-14, 3.56188406200584e-14, -8.28812072946198e-14, 2.9021409606127896e-15, -3.3448803289096904e-14, -7.86816609808899e-15, 2.43192480827906e-14, -3.2773943469161403e-14, -2.73745995410915e-14, -2.8254625026999402e-14, 1.2181349888114599e-14, -7.58529695507312e-14, -1.49915408215673e-14, 1.70099933061655e-14, -4.810356026783111e-15, 5.405598418987249e-14, -8.67067717747575e-15, 2.9958913339749e-14, 1.00046591541485e-14, -3.1227733496861196e-14, -3.35068533499125e-14, 1.24196837822018e-14, -4.22933819121765e-14, 3.90213929060521e-14, 7.196700550013401e-14, 6.60819403376586e-14, 1.9383289061303e-14, 8.42776682757901e-15, 3.01548606417108e-14, 8.594871090269161e-14, 1.19311534966045e-14, 3.0600971839068897e-14, -5.01002491260307e-14, -2.5823458386721297e-14, -5.29711976923862e-14, -1.57221809674333e-14, -4.56194048264843e-14, 4.96054000650896e-14, -1.7359714045994e-15, -1.4792820005993303e-15, 1.90391968133136e-14, 2.6350353062505303e-15, 5.63105316715269e-14, 2.9332606391620206e-14, -2.75470155225534e-14, -7.702480042839791e-14, 7.597442594984309e-14, -4.13572182408721e-14, 7.71754169813579e-14, 1.9203728906317e-15, -4.2921796369460796e-14, 1.10532417476276e-14, 1.01695487405589e-14, -5.4629306648686496e-14, -7.174427810268941e-14, 3.19515257794779e-14, 1.9568272153686102e-15, -5.03983139214355e-14, -8.140828773822898e-15, 3.32127307721122e-14, 1.1589217881680701e-14, -9.546634817246278e-15, 6.35128423669629e-14, 8.421214986224961e-14, 1.12518870999232e-14, 2.9986122950567e-14, 5.3598364813516296e-14, -7.352358721754911e-14, 3.8469854145450796e-15, 3.89400063729559e-14, -5.68952500011402e-14, -2.1350833086680803e-14, 1.12164670841787e-14, 1.25733257853351e-14, -3.8398324804242e-17, -2.49021928938003e-14, -2.3382290349252996e-14, -9.154615629137519e-15, 1.59883461047917e-14, 3.6967386662797995e-14, -5.39448593425108e-14, -9.92051295256831e-15, 7.72666854042018e-15, -7.035110133165859e-14, -2.0722057399405604e-14, -1.58611672982383e-14, -2.25464056390911e-14, -4.6472936478430196e-14, -1.3116571690653501e-15, -9.743043089911571e-14, 6.0918987407605e-15, 1.3073131663200503e-14, 2.1739077111780302e-14, -7.07226499389198e-14, 1.75398362560432e-14, -1.4541920421044302e-14, -3.92631626726539e-14, -1.4365694717713498e-14, 6.446817555684539e-15, 8.02992952817683e-14, 1.5929014213024597e-14, -8.92536648612505e-15, 1.47716664496997e-14, -2.2414403337383498e-14, -9.03533932897589e-14, 9.439009202760291e-15, -1.09414248671692e-14, 1.06792168188453e-14, -4.49302984011051e-14, -6.48636041609068e-14, -5.714615440071611e-14, 3.31656359664865e-14, 6.17248294414948e-16, 6.79684264862903e-15, 3.0069962803064603e-14, 1.23913653525185e-14, 3.7229864360281404e-14, -5.3160210362487e-14, 6.390915854965821e-14, 3.33569993181553e-14, 7.67268206421051e-14, 6.065083550405e-14, 1.2598365381309601e-14, 8.17399207089853e-14, 2.8288040531539197e-15, -7.756959584728099e-15, 3.6050239089797996e-14, -9.812024311895802e-14, -2.63716765161668e-15, 3.8905676851341395e-15, 7.14661918757236e-14, 2.2273963251499502e-14, 6.53436707913541e-14, -2.2449671071552102e-14, -1.5104303481725098e-14, -9.57997097533214e-14, 2.23951493560333e-14, 2.39019668120833e-14, -2.9762603526953003e-14, -1.2070814253042298e-14, 1.68929879129785e-14, -3.21358564620239e-14, -6.762187058632771e-14, -4.55840705271446e-14, -9.768395681574699e-14, 4.2477216236210295e-14, 8.852984495494991e-14, -1.1596917789319299e-14, -5.2406851934119404e-14, 1.3935590896191801e-14, -1.52761121592334e-14, 3.73367584030831e-14, 6.61555527380296e-14, -1.79729917968167e-14, -2.7407646618085398e-14, -9.045212202306259e-15, 2.10876547832861e-14, 5.5858934586059696e-14, -3.3683797182009206e-14, -2.9121937436676003e-14, -8.11132635413578e-16, 2.4858113875028197e-14, -1.4112355899183902e-14, -3.1047101106920796e-14, 5.4233596741646497e-14, -8.245581346488141e-15, -4.34274129046404e-15, 1.74050875152802e-14, -2.18641374183486e-16, 2.6768347225435002e-14, 3.1152782170985e-14, -5.72453797052893e-14, 6.6430663535594e-14, -7.86785146662321e-14, 1.1029950993747901e-13, 4.03986616570693e-14, 5.4138300762726796e-14, 1.2420714112108e-15, -2.45027101589574e-14, 5.2835636393988295e-14, 3.7665605521237e-14, -4.24759349928299e-14, 4.4283676766326604e-14, -6.036553664988599e-15, 3.07545772565767e-14, 2.46426143572097e-14, -5.4048145698668204e-14, 5.16910932586919e-14, 2.7249397713151405e-14, -1.7298359638135396e-14, -4.40105384479189e-14, 4.6445871759548e-14, -9.24529587087106e-15, 2.7586691465307597e-14, 3.8786859600216e-15, -1.62582060861408e-14, -5.71781739000296e-15, -7.220097588364439e-14, -9.32996709769256e-14, 2.64356056601933e-14, 2.1341130679788997e-14, 3.1252366560844797e-14, -2.07805377391064e-14, 4.0208080891347396e-14, 2.4802288187995898e-14, 2.2963516911500697e-15, -2.54800843419617e-14, 1.2734608670740301e-15, -1.38465135249449e-14, -1.1824034013207402e-13, -9.453977168984749e-14, -6.090186497159921e-15, 6.61851682631914e-15, -4.49259762362413e-14, -3.67034210160826e-14, -9.05051330578574e-15, -2.36670484673558e-14, -4.5159478257709996e-14, 7.61225899613061e-14, -5.25642788768007e-14, 3.27982817988116e-14, -9.62212455314662e-15, -4.26546170739162e-14, -8.43678033710023e-15, 5.74858515006229e-14, -3.47650488524315e-14, -8.50206652796144e-14, 4.3814490211029504e-14, 2.7782666957956202e-14, -2.1383916820313204e-14, 3.04615566416601e-14, -2.7143349049218103e-14, -5.31483928177969e-14, 3.63296981845665e-14, -5.28329741207692e-14, 2.8473274617909996e-14, -7.67443439814969e-14, 3.3216890682865895e-14, -2.4150300721296398e-14, -6.3846900510066204e-15, 2.17169488121637e-14, 9.767923912863671e-15, 4.68063539724825e-14, 1.70005010597958e-14, -2.2861497333274603e-14, 4.2302763547306295e-14, -5.229715459722611e-15, 2.4636429182833802e-14, -1.2473941987132e-14, 6.01912519312267e-14, 5.22190214575558e-14, 3.0027503143199996e-14, 7.0616737155433796e-15, 3.752425921223671e-15, -1.45611637652466e-14, -1.3406962446861999e-14, 2.5234899775238402e-14, -1.27580109741051e-14, -7.50531461406335e-14, -5.33865125228743e-14, -3.62126292386683e-15, 5.3482381641419296e-14, -5.60899749068319e-14, -3.1538610310527505e-14, 3.14283745355382e-14, 5.3412865616890204e-14, -3.47038389474769e-14, -3.5228994755847395e-14, -1.88920580066054e-14, -6.23892999462842e-14, 1.25418535387251e-14, -1.15895143155082e-14, -9.95686352323457e-15, 3.83222213752824e-14, -2.12828206266857e-14, -4.411210428656299e-15, 5.62854843593248e-15, 5.81729315013426e-14, 4.2351817157406306e-14, -7.112527134048449e-14, -5.5340521554499104e-14, 4.89109966498533e-14, 2.3927953942755003e-14, -1.27886066391845e-13, -6.10894510021868e-14, -2.12902211734536e-14, 3.04353806255219e-14, 1.6557194863148e-14, -4.80862801718266e-14, 3.6483859026994505e-15, -4.279384737081479e-15, 4.20003206122317e-15, 5.6087681544055e-14, -8.54465966825921e-14, -3.00673157530709e-14, -7.32738846164378e-15, 2.9542528624540105e-14, 2.7404914324718102e-14, 4.8374521009622203e-14, 7.324235680411759e-15, -4.72696416302153e-14, -3.2968513283271e-14, 2.34074519221519e-15, -5.74610241984024e-14, 4.4906525782285897e-14, 7.74691653532215e-14, 3.86211674992811e-14, -4.7977260693294094e-14, 1.58390916163603e-14, 5.3627052492146105e-14, -7.58528308518503e-14, -4.2904202641536795e-14, 2.62258739873026e-14, -6.90586735003204e-14, -3.56594792543405e-14, -4.67992080208181e-14, -1.3027312928094103e-14, -1.28626282126445e-14, 7.14124464295303e-15, -5.1968858653357405e-14, -4.02458611256138e-14, -2.79337441150225e-14, 6.190745798979531e-14, 1.36647179639324e-15, -1.1404034667154101e-15, 2.87593230497238e-14, 5.497539973170031e-14, 1.90385103057728e-15, -3.73469718334422e-14, 4.43763295864979e-14, -1.4868178461323898e-14, 3.52455595383997e-14, -7.76087603708167e-14, 1.8569236473523703e-14, -2.86320898526667e-14, -1.14213612969653e-14, -6.40498001720632e-14, -1.36709402931663e-14, 1.2322653238368901e-14, 2.15988173084585e-15, 3.03646661125033e-16, 1.17368505203407e-15, -8.08527882392007e-14, 2.23134597141038e-14, -2.02183756355778e-14, 4.25702351444921e-15, -2.7585082570391397e-14, 4.92701034201939e-14, 4.66804330287828e-14, -4.2894960246203297e-14, 5.73401793928485e-14, 7.805803817269051e-14, 2.98859316080679e-14, 2.8458499321307097e-16, -6.64285626767635e-14, 2.7998631087504703e-14, -3.49985064047854e-14, 3.2130517999988305e-14, 3.9217261588465696e-16, -3.1614222892904004e-14, 3.71118532226818e-14, 9.13118314634356e-14, -1.8163131998488e-14, 2.6215854775182202e-14, 4.4999342596337094e-14, -7.67233969729702e-14, 1.8363310765691698e-14, -4.3170075873233297e-16, 1.1830390751744601e-14, -8.832045323865531e-14, 1.3257443227720802e-14, -8.77267214136302e-14, 3.6896725278051004e-14, -1.50474230511931e-15, 4.94962817934238e-14, 2.19195398919601e-14, 3.30250371818981e-15, -1.0312271178085698e-14, -3.42913142334284e-14, -8.25501280886e-16, 6.57014163777626e-14, -7.24313326391318e-14, 3.828041881827199e-17, -8.48224292237248e-15, 4.34473006626597e-14, 1.9200261290774398e-14, 2.1658141136201697e-14, -3.99921075360069e-14, 1.15599068414253e-13, -1.0891100905107e-14, 5.51530250732023e-14, 6.58366035022092e-15, 4.08394209244477e-14, -1.3378714138520201e-14, 2.47865110994044e-14, -1.25477228082419e-14, 2.5480057961527602e-14, -2.54665971620405e-14, 5.1605281563017094e-14, 2.84752622107906e-14, 8.80278894940868e-15, -4.1701439836436306e-14, -1.98566738824958e-14, -2.0188696206672402e-14, 1.23996349454551e-14, -1.39034373113348e-14, 7.993606083717731e-14, -6.22189411497009e-14, -6.2582709548423e-14, 4.4379368816739394e-15, -7.750307082056071e-16, 7.33212915238033e-14, -2.1052437649400604e-15, 2.45938417784659e-14, 3.8081593496989494e-15, -2.96116650149776e-14, 1.7905253142544798e-14, -1.0599791047211202e-14, -3.6405602848881005e-14, -4.5521184090777204e-15, 4.34057477544704e-14, -2.4330244058527303e-14, 4.96822429116921e-14, 3.75494226558945e-14, 8.64625582616068e-14, -1.29286221807698e-14, 1.0099796338110701e-14, -1.88722440498468e-14, -4.9653688796691e-15, 8.26523191560737e-14, -1.7839871028714402e-14, -6.3335461467213896e-15, 1.4576974193088198e-14, -4.1118602180739404e-14, -3.7042978791691104e-14, 1.59732752066596e-14, 1.5475113501042802e-14, 8.331311033773549e-15, 3.33994537975923e-14, -7.27450482852041e-14, 8.02951329479369e-15, -2.91563712623847e-14, 8.276862397795499e-14, 1.70532904100907e-14, 2.03352397343944e-15, -2.048651632267e-14, 4.13320893498037e-14, -6.17483631166812e-15, -3.1237572296381998e-15, -4.0459617344486206e-15, 6.91356419921212e-14, 6.01870866668002e-14, 8.42171334161074e-15, 5.0444996773148803e-14, -1.8940880265919197e-15, 5.6345871871118297e-14, 7.048591451272891e-14, -1.27482568968061e-14, -1.9714663903322602e-14, 6.413052017796041e-14, -6.32378750631762e-14, -4.53127664485445e-14, 2.49005389038493e-14, -2.95519057302293e-14, 4.1434051217943494e-15, -7.383483248591381e-14, 7.23620775105817e-14, -9.11354171306618e-14, -5.42471533518893e-14, 3.89405578539471e-15, 3.620397840509941e-14, 4.17720418437852e-14, 2.88990482499012e-16, -5.89771391694087e-14, -4.11114476641044e-15, -2.93899090672796e-14, 1.76566507731418e-14, -4.31993663128551e-14, 2.92281867997571e-14, 4.04580285061746e-14, 2.1296723535325e-14, -7.561808024286389e-16, 1.39963663875176e-14, 3.20616506643971e-14, -4.44373701978613e-14, 4.16870755913631e-14, -3.25030271901498e-14, -2.06766027674023e-14, -3.2540247680643104e-14, -1.26453608954601e-14, 1.3879033739244699e-14, 6.63343659234814e-15, 2.39372603849982e-14, 4.69363867788944e-14, 1.3611501401498799e-14, -4.9210932720839e-14, 7.163126115141271e-14, -2.43815331287381e-14, -7.54317186414654e-14, -3.71990021832492e-14, -1.18348365454222e-13, -5.74899969126779e-14, 4.27806288979193e-14, -1.13422518760784e-14, 5.54862360799278e-14, 1.67096938130079e-14, -1.82614719501904e-14, -3.5065637160214405e-14, -6.2186198929289e-15, 1.78914954419388e-14, 4.20485591437264e-14, 5.5941275972677806e-14, 8.37054873061475e-14, 1.8785811745544497e-14, 4.2546239338804394e-14, -1.28214355801579e-14, -2.38990109561511e-14, 2.74897123428764e-14, -5.76405332594215e-14, 2.93632968304607e-14, -4.3905378146987294e-15, -3.36409330525977e-14, 3.14482167346637e-14, 1.24136953545029e-13, 1.61814125362968e-14, -3.52398997548115e-14, 3.16628813006092e-14, -4.985244664855929e-15, 8.714267253038609e-14, 6.556712792219059e-14, -2.1024913100532e-14, -2.2326897199480398e-14, 5.887533283612169e-14, -1.0621265643067499e-14, 2.64483341329749e-14, 3.44387648456789e-14, -1.15444938834448e-14, -2.2996900070851002e-14, 1.4628097600679e-15, 5.92703337799681e-14, -3.48259359512152e-14, -8.75457773474494e-15, -3.2775020723315103e-14, 3.64989183892091e-15, 1.45008744891343e-14, 6.6000984203522e-14, 8.08034731629575e-15, 3.96273459205579e-14, 1.3857367310643e-14, -6.6684322329528994e-15, -4.607802263707981e-14, 1.97608501900979e-14, 2.9477528582340895e-14, -1.77975693324897e-14, -2.1515914275087e-14, -2.29101600265266e-14, -3.42466229971564e-14, 1.1220587849388901e-13, -1.1179881105087899e-14, -1.3792443235916099e-15, 4.11910892468449e-14, -1.61061721953209e-14, -4.96987631971268e-15, 4.15250297838524e-14, 2.5159098665084e-14, 1.2542358927789199e-14, 4.0050469568013e-14, 3.01902678764737e-14, -2.1162448023649103e-14, -5.57809272905909e-14, 3.79930199483143e-14, -3.4900000449923897e-14, 4.30856407535177e-14, 5.256897837254901e-14, -5.20324159868669e-14, 6.58033672223382e-14, 5.59387806944657e-14, 6.17117411753797e-16, 6.50570175947364e-14, 2.8175584568015e-14, 4.9222243073403297e-14, -4.7111561013166496e-14, -2.21835095375468e-14, 6.59417661576021e-14, -2.1856962835155102e-14, 1.73077189366664e-14, -4.96311653189266e-14, -3.19899361526755e-14, -4.10795238976319e-14, 5.14571283879075e-14, -5.30688778521518e-14, -6.621469510064e-15, 2.4791991228748698e-15, 9.518579882353529e-15, -6.81093214338685e-14, 2.96537618737314e-14, -2.40682581347319e-14, -1.78239733727951e-14, 3.21032303499729e-14, -3.18363235502325e-15, -7.893482663199291e-14, 1.67686129753759e-14, -2.26595549150187e-14, -4.95933724742055e-14, -4.39174174041449e-15, -6.73044978712029e-15, -3.76556583541121e-14, 3.7862313268879405e-14, -6.602796437229079e-15, 6.064910634864679e-14, -8.153633731112739e-14, -1.2990428361366801e-14, 2.2996735183208902e-16, 1.16094927626868e-14, 2.32210639382354e-14, -2.86829253724676e-14, -8.002739432055589e-16, 2.99949682919924e-14, 4.52660418507441e-14, 1.37022756778825e-14, 3.4393355386968195e-15, 3.7514001139105304e-14, 3.74339725534615e-14, 1.44466842992502e-14, -1.87042050256741e-14, -7.20433979803066e-16, -1.3586663926288299e-14, -6.79487604895088e-14, -1.1867328758355801e-14, -9.213005412486841e-15, 5.701191757862901e-14, -2.61201422291415e-14, -3.60737089381379e-14, -3.5933127879336407e-14, 5.5563907317593806e-14, -1.19554070294182e-14, 1.2166147287036301e-14, -4.78502016302653e-14, -2.42773735245779e-14, -2.40791324562412e-14, 4.3154450171781503e-14, -5.493338346191999e-14, 2.6620697056976e-14, 5.20747641293485e-14, -7.59931155931323e-15, 4.4162745921799605e-15, -4.85865167009637e-14, -8.34160092914736e-14, 5.945337686426601e-15, -4.4280581601558705e-14, 5.56198869140847e-14, 4.3210173563202005e-14, 1.25176527875329e-14, 3.74707898664709e-14, -5.24486538172105e-15, -6.367755621304971e-15, 1.78353533309152e-14, -8.583836899387829e-14, -1.8763314302698002e-14, 2.34234908188967e-14, 5.35272516756964e-14, 5.70783797229413e-14, -1.6304858885683298e-14, -1.26930537248813e-14, -2.6475855795204104e-14, 1.6557107530469198e-14, -5.00299045816756e-14, 6.14004183153255e-14, 5.78155470995013e-14, 1.64628360449723e-14, 1.7748403574972396e-15, -2.19759823955729e-14, 1.75682024148035e-14, 6.23069135165641e-15, -2.8791214492189206e-14, -3.9908265469637e-16, -3.12153209378261e-14, 1.5886791813416302e-14, -1.39496737040721e-14, -7.64708538824779e-14, -2.83482967406488e-14, 5.30457083749979e-17, 6.2976504524471e-16, -4.6808184304928e-14, 1.00952760774711e-13, 5.49693611931458e-14, 9.465721038134242e-15, 5.39590101375032e-15, 2.26903504691426e-14, -4.2242726429033704e-14, -2.9924468171669604e-15, -2.07700685744006e-15, -1.3932422093298502e-14, 4.11277837766908e-14, -7.40011265744564e-15, -3.49094124955444e-14, 1.61486219558665e-14, -4.14016071273373e-15, -8.26591588197577e-15, 1.8131072217880902e-14, -1.89827039431246e-14, -2.72178087426234e-14, 9.696273848012499e-14, -5.02686249714857e-14, 8.47851073707314e-15, -3.4501758991103e-15, -1.7655063947725598e-14, 5.35937879201053e-14, -3.8909082002929706e-14, -1.14720763291059e-14, -2.6797780054411298e-14, 5.62704272168666e-15, -7.82800063459902e-15, 1.49381497672023e-14, -2.15366326594698e-15, -4.261076015102299e-14, 1.92559503011744e-14, -1.02719189218761e-16, 5.85302585921341e-14, -5.6496265580789e-14, 3.59371509671657e-14, -4.71835250156735e-14, 4.2636650397590995e-15, -3.3503400773918e-15, -1.48222321862829e-14, -2.22191931306057e-14, -4.43996636439751e-14, -5.3073030769582e-14, 5.875746590746881e-14, 4.82863419057219e-14, 3.9103174469208e-14, -5.609570242652871e-15, 3.50936019951615e-14, 1.0651412829102498e-15, -2.21896034944006e-14, -2.7101971132019203e-14, -3.37138431730831e-14, -1.1310243638142598e-14, 1.34729664654283e-14, 3.76963517756481e-14, -1.00984796256516e-14, 3.2480359872491e-15, 3.43679571069132e-14, -2.61926594022435e-15, 2.8493506856989e-14, -1.09560851391912e-14, 7.161188684395621e-14, 1.13094825915431e-14, 5.97135198866192e-14, -5.85304944801573e-14, 1.02165205066137e-14, 1.99923292514059e-14, -1.07377519953669e-13, 9.736529670598649e-15, 1.34964876246602e-14, 5.1359214451755704e-14, -1.53031704422441e-14, -1.88923740426911e-15, 2.57125158570662e-14, -4.4574869032015504e-14, 3.80956751101632e-14, -8.47844145095977e-15, -2.8918049352469e-15, -4.7646172283762095e-14, 2.07732321207324e-14, 7.01349457106364e-15, -4.431707810745271e-15, 3.20235649142413e-15, -6.474782877732601e-15, -1.74407382030863e-14, -6.4716378162357e-15, 2.4000747545595002e-14, 3.74197150909621e-14, -2.90891745284976e-14, 1.5452907561904199e-15, -5.0934859552756104e-14, -4.84564987064853e-14, 1.0685762468319599e-14, 3.4595507297134804e-14, -3.8644977303113894e-15, 3.82113223037695e-14, -3.32111493931529e-14, 7.952291663191841e-15, -2.66175634984888e-14, 3.68917324151481e-14, -2.09267427359122e-14, 6.445254100559221e-15, -1.46450994641963e-14, 4.30060666866753e-14, -3.2500368756589395e-15, -5.35024358068562e-14, -4.49251750046059e-14, 3.8803206802366e-15, 3.10675556855044e-14, 1.83568919645069e-14, 6.089413699284401e-14, -1.48523829250712e-14, -8.366336568213031e-15, -3.59004336295416e-14, -3.52415566518244e-14, -1.4090570119224799e-14, 6.85670980279882e-14, -2.9931680752193804e-14, -2.15008835690107e-14, -1.78366145150923e-14, -1.3493247204051202e-14, -3.25566797977664e-14, 2.5730347367253398e-14, 4.95874849604807e-14, -3.6440115537662304e-14, 3.9142961337777897e-14, -3.95866337226876e-14, -4.40860670771185e-14, -8.648533568110648e-15, 3.7915082828479595e-14, 3.35191970702424e-14, 1.78952242204736e-14, 8.94188928600873e-15, -4.5945702784163104e-15, -1.87868505816728e-14, 3.604166889009681e-14, 6.13592428567512e-14, -1.47888307246564e-14, -9.598739207210291e-15, -5.0530469417880205e-14, -3.69129278731802e-15, -3.39202250526675e-14, -3.6630950349892303e-14, -6.11769808163144e-14, -5.82049167552139e-14, -4.31291578168141e-15, -2.31907128303699e-14, -6.37740325606371e-15, -3.18875017369192e-15, 5.1113907940752e-14, -4.16083251797331e-14, -3.0849194517403003e-15, -9.3628743546129e-15, -1.89484402131375e-14, -4.0472261084798105e-14, 8.3105024731609e-15, -2.2605250278838902e-14, 4.3109093600494904e-14, 3.51519852513017e-14, -3.2707686642982904e-14, -3.32719050703459e-14, 3.57981495570217e-14, 3.6727741579949e-14, 2.3191770821299197e-14, -2.9182293398753996e-14, 6.73449833644583e-15, -1.8221277774002102e-14, 2.8347061636819397e-14, 6.56931193881682e-15, -4.5573757384294504e-15, -3.619164242199579e-14, 1.08771229789503e-15, 1.32101930934536e-15, -3.5738787395809396e-15, 3.77188293682729e-14, -3.42901918394698e-14, 2.9596143813840604e-15, -5.35765787017261e-14, -3.34746506638924e-14, -1.81965640625232e-14, 1.54056681855119e-14, -1.83074713577505e-14, -2.13242985946347e-14, -1.9726340630918202e-14, 1.21167999346247e-14, 5.5873445719395396e-14, 4.08013422903591e-14, 1.51079516470636e-16, 2.6948661751128902e-14, 5.90141547613596e-14, -4.6338399569160403e-14, 1.7174802688026698e-14, 2.66977206844948e-14, 5.40026374082959e-15, -2.6256374417448997e-14, -2.80591672186807e-14, 3.3135041558317695e-14, 4.29598703346753e-14, 2.45212262944772e-14, 2.4997932002092202e-14, -1.68255916622201e-14, -1.8885213588661303e-15, -4.80069553072797e-14, 2.38182933531022e-15, -2.5278264867746798e-14, -2.9914848837112996e-14, -4.42856892894201e-14, -2.1496385897022502e-14, -7.111850232843041e-14, 1.92769529863591e-14, 1.12384869916253e-14, -1.6707676246208298e-14, 2.54646065293391e-14, 3.04162303721442e-14, -3.4804040001471395e-14, -6.354279466762181e-14, -1.05946035271749e-14, -3.2594181708257105e-15, -8.43915608860882e-14, 5.40505344655518e-15, 4.16632614689316e-15, -9.949177848085969e-15, 2.69593990746757e-14, 7.550232267901849e-16, 2.52246922539816e-14, 1.09259608906357e-15, -5.14950345409925e-14, -5.765575823399649e-15, -2.55383912563922e-14, -8.140980245339811e-14, -6.60236709916854e-14, -3.14260417056694e-14, -4.60326001584646e-14, -4.42261034557093e-15, 1.53733311374717e-14, -4.2402910502474194e-14, 1.98241713328255e-14, 1.61440356301826e-14, 4.8002588281242896e-14, 4.95406078164571e-14, -6.08385169798959e-15, 5.927295004543889e-15, -1.49652115401387e-14, -1.9687252351698998e-14, -1.44387465011433e-14, -8.49827708174914e-15, 3.3674178472902204e-14, 5.0479102139626e-14, 4.9172481325371595e-14, -3.0650517370276796e-14, -2.7965146011083e-14, 9.74200980830311e-16, -3.21069639655367e-14, 3.13530868258832e-14, -3.4660233028569604e-14, 4.56703122864956e-14, 7.2365732179875e-14, -9.419749390465759e-14, 4.46438723101674e-14, -1.58246737820281e-15, -1.85837979813668e-14, -1.99263823546684e-14, -4.2867325007444596e-15, -2.54435339232014e-14, 2.7942552572869e-14, -1.79000252767202e-14, 4.7368183103311796e-14, 4.7153631995748995e-14, -2.91677438185219e-14, 1.61313145886446e-14, -1.15972836685217e-14, -4.11878892140093e-14, -5.150808509542289e-14, -3.84314218471453e-14, -6.95343741773679e-15, -2.0482631893661003e-15, 3.3526854285351197e-15, -3.05722122230726e-15, -2.88285727919446e-14, 4.857124086172799e-14, -3.7945209983373806e-14, -8.145199913469909e-15, 3.3372050201263406e-14, 2.69983089140689e-14, 1.94141921529514e-14, -6.16556173613367e-14, 9.33172639524947e-15, 3.56713715064956e-14, -2.69085377494347e-15, 3.1373334941344294e-14, 4.66469546641453e-14, -3.76714000331028e-14, -1.8696148615698002e-14, -3.09137479430991e-14, 3.2557628558907194e-14, -1.74818820339814e-14, -5.76916093892906e-15, 3.882468306397429e-14, 2.3755905689581403e-14, 2.06315308575851e-14, -6.5372861131241e-15, 2.2048810770967698e-14, -3.86643387460354e-14, 2.51635762331987e-14, -9.82237878956639e-15, -2.2761695856144597e-15, -6.1686534085094e-14, 7.74598237439718e-15, -2.49607860598882e-14, 3.62691291035539e-14, 2.26538633915498e-14, -1.9445754666074703e-15, 9.07146991977487e-14, 4.0716059894587097e-14, -9.746043318003851e-15, -3.81635443448034e-15, -1.1769363880879101e-14, 1.4053931684628e-14, -1.20080840744563e-14, -2.7470020528567905e-14, 1.49578906054457e-14, -2.61116798605381e-14, 1.45547018446935e-14, -4.05422483195736e-14, 3.3128623475844605e-15, -4.71303853162822e-14, -1.1071681210746599e-14, -1.0656417769874202e-14, -6.5153805450243794e-15, -6.756718472333081e-14, 3.0439683031665496e-14, -4.1352292448475704e-14, 3.24734350528084e-14, 1.6573127884549898e-14, -7.53314086859631e-15, 2.03644038537402e-14, 3.10624711096979e-14, -2.19431939421026e-14, -7.26384007875119e-14, 3.09662187930699e-14, -2.5267144532573997e-14, -8.76499698246471e-14, -4.65960141489362e-14, 1.57762652820175e-14, -6.02868984894071e-14, -3.59922723620789e-14, -6.5767887900656004e-15, 2.7181963056157797e-14, -1.01496407940099e-14, 3.7036859386338e-14, 1.73037980965183e-14, 7.12530637214845e-15, 3.39790360101375e-14, -6.150906121145321e-14, 2.04733694613119e-14, 3.5125771308294297e-14, 1.44059108189606e-15, -1.26958870863503e-14, -1.8944280988684202e-14, 5.20262346510954e-14, -6.1280626286676e-14, 1.2586520689095801e-14, 1.58231712017232e-14, -2.1134902604902598e-14, 1.5675083440142002e-14, -6.99266095291478e-14, 1.16152099624533e-14, 4.0403946957043e-14, 7.44310778526233e-15, 2.8011142131557596e-14, -9.561614849222039e-14, -7.21614486557047e-15, -2.45181051366661e-14, -2.44182183380242e-14, -7.6503584154844e-14, -1.10505053011581e-14, 9.300747959151221e-14, -1.0830118473572901e-14, -3.4524046640591806e-15, 2.9659026527494597e-14, -6.43969816163629e-14, 7.76862958227441e-14, -5.722467379300019e-16, -3.27163638355582e-14, 1.1207097122947201e-14, 7.57702392178871e-15, -5.3229099613397694e-14, -4.93730631704227e-14, 2.1429727859479697e-14, -1.02595060952738e-13, 8.18465462408961e-14, 3.2298631256035705e-14, -1.31534670064512e-14, -1.3151022390641699e-14, -6.858616262370339e-14, 3.83642062822146e-14, -1.1960514612151301e-13, 2.65142162560739e-14, -1.08389953237119e-13, -7.990221745412429e-14, 1.26095398985891e-13, 5.17366190304302e-14, 1.1337052220204501e-13, 8.959033654568821e-14, -7.380277175797951e-14, -7.0906017822266e-14, -4.611829909718401e-14, 3.27712102426679e-14, 5.7455851204886706e-15, 1.3585615550454099e-14, -8.61626396265328e-14, 1.13929929942424e-13, 8.37324824363783e-14, 1.02888545574345e-13, -4.90723438590646e-14, 4.13223644420486e-14, 1.00913796940762e-13, -7.66141476443038e-14, -8.98635582904492e-14, 5.87532628930621e-14, 2.1981886003046397e-14, 1.10157779362907e-14, -1.6845077522756203e-14, -1.1315842034066799e-14, 2.79139661992718e-14, 3.61398400881634e-14, 3.4128296303388697e-14, 9.72803903253481e-14, -3.62519770382502e-14, -3.85354047638465e-14, 1.1027328547008301e-14, 1.7891594355450002e-14, -5.10764042798679e-14, -1.7649170639232798e-14, 8.8357517605597e-14, -1.28615949000467e-13, -3.98023759515956e-14, -1.06777094904928e-13, -9.2907958830421e-14, -1.0676169269813001e-13, -1.84932316346979e-14, -1.27137216489293e-13, -1.05896921886777e-13, -7.08266932341061e-14, -4.14896229946588e-14, -6.51248781204e-14, 8.17130419917618e-15, -1.3519616262231897e-14, -7.6106820546663e-14, 1.16117289949804e-14, 5.64528263588128e-14, -2.2891962187130098e-14, -2.25748399217677e-14, -9.4501154356681e-14, -1.49826149426072e-13, 5.09936430804828e-14, -7.49551060206358e-14, 7.34889851284109e-14, 5.89055488958137e-15, -4.877358906926099e-14, -8.582175534260119e-14, 4.8896420185218696e-14, 2.3961081231910598e-14, -6.07045028006482e-14, -2.6920382698058e-14, 3.68770862223611e-14, 2.5893942192227003e-15, -2.06036795032329e-14, 6.69576810647189e-14, 2.16569718621848e-14, 4.33486067514593e-14, -2.9757298179715597e-16, -2.06519165609702e-14, -3.73848984774234e-14, 3.492801190497701e-14, -8.603809727576981e-14, 1.1203084159384101e-13, 2.6900671955312696e-14, 1.4044462930626097e-13, 1.0181838825357302e-13, -3.32354510592531e-14, 1.19645288591916e-13, 4.33246843023792e-14, 5.508826334649569e-14, 5.49269087306954e-14, -4.7509905958750705e-14, -4.03429816770374e-14, 1.23963879951731e-13, 1.03613248467682e-13, 5.86816126584789e-14, 1.3164727179659802e-13, 5.68006447552174e-14, -8.944245797726221e-14, -8.913400739021671e-14, -5.985407512397109e-14, 2.60640689097752e-14, -2.00942489206874e-14, -6.22488636560022e-14, -5.55669208355838e-14, 6.03981101336561e-14, 1.05187290941304e-13, 8.922356079976599e-14, 3.06753725371896e-14, -6.583834385466709e-14, 2.27689139079245e-14, 4.64515057315301e-14, -4.56294232545065e-14, 1.7024884106151301e-15, -2.4053019204670702e-15, 6.411724656047161e-14, 4.23423277282199e-14, -2.13089257803624e-14, -3.41545639264135e-14, -1.3012667926924501e-13, -1.0284944650357699e-13, 1.8329130638403198e-13, -8.85817063821098e-14, -1.24268081541922e-13, 1.27040098146888e-13, -7.868728705208141e-14, 5.04956529455095e-14, -7.67579073513808e-14, -1.23516658424192e-14, -1.40842098623905e-14, -2.2466607653306802e-14, 5.56099182269685e-14, 1.2101116576535802e-13, 6.65620906490966e-14, -3.60372684464991e-14, -7.169126646344981e-14, 7.00700581149234e-14, -4.87870765824644e-14, -5.503077710816369e-14, -5.842903706933071e-14, 4.74926943865078e-14, -3.34197299573885e-14, 3.30410604777076e-14, -5.80291644556835e-14, 3.1533617726620295e-14, 5.04142268168562e-14, 7.82738873749778e-15, 6.18414626983726e-14, 6.0632879415246e-14, 4.9572080214519996e-14, 8.71697804244124e-14, 1.81437211045234e-14, 1.96860308269976e-14, 1.3470371156575999e-14, 5.30414279757749e-14, 6.66517450981119e-14, -4.00318123753358e-14, -8.66135093528023e-14, -1.00888937773531e-13, 1.1980693666554e-13, -2.9569945092701e-14, -7.46009448965295e-14, 1.49637288286884e-14, 8.407314374815399e-15, 4.06370075007959e-15, -6.83604758608303e-14, 5.3246292072837e-16, 7.059161281734841e-14, 5.84120399809129e-14, -7.17650781886147e-15, 5.94763893685553e-14, -7.51404866679536e-14, -5.572015614561381e-14, 5.72157749827824e-14, 4.7991781234483906e-14, 1.10912443189025e-14, 2.4347175386184397e-14, -3.06157411500769e-14, -9.045365428950452e-15, -2.0261801435432e-16, 2.41178437872028e-14, -2.47360053081586e-15, -1.02807368231262e-14, -5.2901021087286493e-14, -3.93418124886497e-14, 6.727006433278279e-14, -9.157763739311909e-15, -7.889172337585671e-14, -1.19100588635894e-13, 5.76489748570481e-14, 1.04171315073091e-14, 3.65501629170441e-14, 2.16229928055333e-14, 2.77417846247575e-15, 1.75382700626789e-14, 2.23025978815804e-14, -4.0679495600314496e-14, -7.892627754621341e-15, -1.47512141013534e-14, -2.16872209245596e-14, -2.21177778524663e-14, 4.39387345263277e-14, -3.88097980172556e-14, -8.7322008253562e-16, -3.73102940029214e-14, -9.828751555490598e-14, -4.0929068640839404e-14, 4.2319454790463e-14, 1.1845062309827903e-13, -7.924397022136458e-14, 3.4499151801675205e-14, 8.5982272546366e-14, -2.3945745585686198e-14, 3.61472029253488e-14, -1.77655319786268e-14, 7.79172381125631e-14, -2.29439092348585e-14, 6.76093070860164e-14, -6.29188258687616e-16, 3.14327128267381e-14, 3.17826433483032e-14, -3.6733451668576204e-15, 6.40948500561308e-14, -4.67568588861085e-14, 4.88896874339161e-14, -1.17062330125937e-13, -1.46335711963196e-13, -4.96534331825504e-14, 6.71573035218458e-14, -3.98911398928009e-14, 5.692290607812541e-14, -6.14536770008419e-14, 3.3914006353105497e-14, 3.5746665181936007e-14, 1.06763112956989e-14, 2.44319884566274e-14, 6.86876503815711e-14, 2.0513156296734904e-14, -6.75829637244334e-14, -1.8681823836524502e-14, 2.26320420497693e-14, -3.6219009261390895e-14, 3.17380130513433e-15, -2.75347913028105e-14, 2.89757249039028e-14, -2.00641115390268e-15, -3.27460884844374e-14, 2.7749842518935203e-14, 9.58889106696249e-14, -2.2181502784324404e-14, 2.9800445916150906e-14, 7.851007762118191e-14, 3.93595365811262e-14, -2.8713535348057005e-14, 2.4586864683079396e-14, 3.84096646576603e-14, -8.71335363192097e-14, 8.38420680270974e-14, -1.16914565164104e-13, -9.38731316262879e-14, -5.44666783048791e-14, -4.67635612486423e-14, 8.419364499256959e-14, -4.12915471186351e-15, 5.98597852412584e-14, -2.94263573241921e-14, 4.7822006498609705e-14, -1.0484886111595802e-13, -6.41225192569903e-14, 8.443449580398159e-14, -1.0520575019003801e-13, 1.35000259243981e-13, 1.54853010285906e-13, 1.3655916151860902e-13, -6.18907441111302e-14, -1.02426787526484e-13, -5.6447165533919994e-14, -4.44346882965432e-14, 2.7315190788960603e-14, 1.63588711681189e-14, -7.751839223425831e-14, 1.50459294952497e-14, -4.3348736796673605e-15, 4.33603651928129e-14, 9.521494683702721e-14, 4.65551114018487e-14, 8.80433912233953e-14, -6.57529965822858e-14, -1.9109781091112202e-14, -7.942099766911469e-14, 7.943894427496941e-16, -2.41777011839939e-14, -6.2684484933442e-14, 3.24676633946143e-14, 1.25351679793474e-14, -2.5044905034199102e-14, -2.17297400683052e-14, 4.7066652848407705e-15, -1.21993474168821e-14, -4.99802852749374e-14, 4.4736827021238905e-15, 2.5371175679090598e-14, 3.91916737758095e-14, 1.01629419494226e-14, -2.39426016906463e-14, -6.11617695253827e-15, -5.70181447597001e-14, -2.97048362825415e-14, 1.9760823013282798e-14, 3.00295511351025e-14, -1.3059993558038399e-14, -2.9284867563521706e-14, 9.03952758974998e-14, -1.4094378140443399e-14, -3.5268835418425796e-14, -1.10457594444715e-14, -9.78225228310729e-15, 3.98163890613566e-15, 5.29212794537059e-14, 8.31731888796147e-14, -2.0273478159840398e-15, 1.0983517896634901e-15, -7.560638862990659e-14, -6.7991504916510495e-15, -1.1489131242804902e-13, -6.03914585679204e-15, -4.04914667036781e-14, -3.06726196889371e-14, -1.4136648426858e-14, 6.650464731087861e-14, 6.8517055979742094e-15, 3.0107080975881e-15, 6.5810369577248e-14, -3.34547885016941e-14, 2.9065572643193806e-14, -3.2791226992078e-14, -3.84801245808404e-14, -4.96420668665544e-14, -4.4701544996573404e-14, 1.9550602460159304e-14, -8.11416333375373e-15, -7.59125144985594e-14, 1.1257153073938901e-13, 2.14891621119753e-14, -5.27559558381707e-14, 1.6344067380572703e-14, -9.04094608297661e-15, -1.0646857786672699e-13, 2.52073243409188e-14, -4.2627384464185505e-14, 2.53389431437462e-14, -8.30219820720311e-14, 6.55601375875044e-14, 1.38548868235951e-13, -4.95319552270887e-14, -8.11307470439067e-14, 1.1655142609137102e-13, 8.54404408206703e-14, 1.25228037936752e-13, -1.01526397899878e-13, -9.339260588438199e-14, -8.23908860857477e-16, -2.3677327881148103e-14, -7.75569863988464e-15, -5.973506921706541e-14, -5.983610194801989e-14, 7.67202670243698e-14, -1.38036732744879e-14, -2.81593235689173e-14, 6.19566795418722e-14, -7.252524940907989e-14, 5.98161941068443e-14, -1.2257960659918598e-13, -8.821168092891659e-14, -1.46657326303151e-13, -1.31778854256615e-13, -1.7017495727576e-13, -1.5836964342341e-13, 1.63114433107368e-13, -1.39207481116695e-13, 8.10059868309936e-14, 6.50068154458629e-14, 4.5537435049550204e-14, 2.0507655511422502e-14, -2.8457723845813e-14, 4.62228006226233e-14, -1.1092123952897302e-15, 9.25012516772787e-14, -9.976726615650219e-14, -5.63993537832941e-14, 1.5968455815382299e-13, -1.36145724333081e-13, -6.296525242683121e-14, 1.13806460774609e-13, 1.37461155686207e-13, -2.27204680098488e-14, 5.86855872254439e-14, 4.2646418390842904e-14, -8.015933089021209e-14, 1.02026954810077e-13, 1.2511819366694198e-13, 1.5459355233139e-13, -1.42532250751257e-13, -1.25993358901666e-13, -7.54028555532933e-14, 1.0438661953493e-13, 9.77524423294356e-16, 6.0827664918542e-14, -1.3688171544257901e-14, -1.26142331004283e-14, 5.395548419425779e-14, -7.84082886015288e-15, -3.38741239301284e-14, 6.84763082422337e-14, -2.78889223186454e-14, -6.81964437463022e-14, -1.02150020932922e-13, -1.2056845934368902e-13, -7.034529567477209e-14, 8.11334221509168e-14, -8.758573034303751e-14, 2.6096005829983603e-14, 4.6706089808208004e-14, -9.98771395507345e-15, 1.70359713433382e-14, 1.68636692738576e-15, -7.73758065996256e-15, -4.65612008027225e-14, 4.27893178070523e-14, -7.072893258497051e-15, -4.15999269284518e-14, 3.54008565884549e-14, 4.9807402709682903e-14, -3.64322406234405e-15, 2.0414796029162202e-14, -3.18066231232262e-14, 8.22146249245731e-14, 4.81863180667892e-14, -3.25758363994659e-14, 9.49150301853916e-14, -1.9716725191747196e-15, -3.779170689585029e-15, -9.997226646051479e-15, 5.21063941469328e-14, 3.91642820202989e-14, 6.12454483780726e-14, -6.583716187279011e-15, -9.22073618349156e-16, -4.81152993565572e-14, -5.2338236392721606e-14, 6.37963126352746e-14, 1.29202009519454e-14, 2.06328435485439e-14, 4.8475747892399197e-14, 6.49529355705958e-14, -5.3789091307335296e-14, -5.1851866469155104e-15, 3.27285433437613e-14, 1.52202954774394e-14, 7.38808129947271e-15, 3.807797545432e-14, -6.472144213316591e-15, 4.11665914296765e-14, -1.9065827145277998e-14, -2.2096609208963798e-14, -2.6628760333977102e-14, -7.828159922871311e-14, -2.89788789360754e-14, -1.1292739026352901e-13, 6.22413414932102e-14, 9.52813392865547e-14, 6.742104025377e-14, 9.78933696430181e-14, -3.48545530840064e-14, -1.73398495193286e-14, -1.3715796332705599e-14, -2.3331370846193097e-14, -2.9917165472027594e-14, -4.22067584053806e-14, 4.72747939191826e-14, -4.2085347102602896e-14, -9.1586387746256e-14, 1.6980014012298301e-15, -1.62386415079784e-14, 2.6801373995176203e-14, 3.64749397674121e-14, -6.42020241842321e-15, 8.17572772781957e-15, 1.6878406563494697e-14, 3.6152219596144e-15, -5.5703910401824895e-14, 3.78152535815941e-14, -5.02721356733969e-14, 3.22472200583056e-14, -5.4549839675748294e-14, 6.432393609760331e-14, -9.680737808225829e-15, 7.842223658385471e-14, 2.09176396989875e-14, -3.7397086052050694e-14, -3.6113950939227404e-14, 8.37377902855611e-14, 5.258894835268329e-14, -3.0259227319328305e-16, 1.42278219501766e-14, -4.8801399185521504e-14, 3.7571806229799e-15, -1.85946483171874e-14, -4.7303491429085e-14, 6.71880101605929e-14, 3.71082805443096e-14, -3.74397412193608e-16, 4.5743924374337897e-14, -4.01759837107107e-14, -3.5263529119591005e-14, 3.1261746665142005e-14, 6.8679481071804504e-15, 2.31940740588293e-14, 8.94755901471359e-16, -1.4181671486577e-14, 1.12665908320366e-13, 6.14972574071445e-14, 3.5379547233212896e-15, -1.4026816490143199e-14, -3.53366154169626e-14, -4.4525640214980396e-14, -5.3798393465662e-14, 7.88350151793454e-14, 5.46306790668316e-14, 6.58228574689893e-14, -5.28945794676975e-14, 6.44826250134631e-14, 1.09573134794935e-13, 1.0442235359574002e-13, -1.53698634510361e-13, -1.1190062415636e-13, 5.83882615347551e-14, 9.00876186106117e-14, 2.5991839672025902e-14, 1.64625398671989e-14, -1.8435258242365002e-14, -4.0208725729250803e-14, -3.600402049674131e-14, -3.95797448406255e-15, 1.1333216384638799e-13, 7.99104031667481e-14, 1.36985467663194e-13, -1.45996714869192e-13, 9.12615581718851e-14, 7.9495471113315e-14, 1.1099318845887101e-13, -5.50084201866342e-14, -6.8003102642403e-14, -5.91042958030622e-14, 1.2253579955284599e-14, 6.67568697049001e-14, 1.18132713580848e-14, -8.70443851098778e-15, 6.03483281395169e-14, 4.2266278031731506e-15, 1.71462467164619e-14, 2.39360275334101e-14, -6.9753408620146e-18, -1.07310040541175e-13, 9.302730218536391e-15, -1.0998452609825199e-13, 3.9222647201446404e-14, 7.573756774777981e-14, 6.958438288013659e-14, 8.087127724720559e-14, -4.6030297817834704e-14, -8.01182428631686e-14, 2.27980226461445e-14, 3.6723994613877006e-15, -2.00933855856473e-14, 2.67329450888057e-14, 5.30971808870622e-14, -7.85647461518078e-14, 3.846453178724019e-14, 1.20443887545648e-13, -5.4533352755968496e-14, 6.455697993728501e-14, 3.2512577901627e-14, 2.64031110513251e-15, 4.94605038276211e-14, -1.04540914104202e-14, 6.98104700931647e-14, 4.1505872083233795e-15, 3.51352782777731e-14, 7.67369492607437e-15, -6.42720416026439e-15, -3.6479658783875504e-14, 1.13734726347108e-14, 3.00634115481048e-14, -1.7806339974258703e-14, 2.56281969136231e-14, -4.9125600881378895e-14, -8.369746100292281e-14, -7.99042561665307e-14, -1.5138790507374302e-14, 4.1899431967811204e-14, -7.632112118845401e-14, 6.12317991034699e-14, -3.96925248244446e-14, 1.3117361280150001e-13, -3.07864503180178e-14, -4.218606033532361e-15, 6.35546496140484e-14, -4.07029576026713e-14, -2.3247462756290698e-14, 2.7114612494293898e-14, -5.945008004488231e-14, -3.27064459284705e-14, 1.39481073068747e-14, -7.223494113842011e-14, -6.64381349825426e-14, -1.95029286513582e-14, 1.9800478201069696e-14, 2.68398556722982e-14, -4.26450505206854e-14, -5.00832996035242e-14, -5.26510635534776e-14, -2.55751417914609e-14, -5.21640085844087e-14, 8.09677342044861e-14, -6.357390921425799e-14, 4.1893658167101095e-14, 1.01656818185973e-14, 2.4909369442216202e-14, -2.95349962914932e-14, 6.80454064510002e-14, 7.98375587404602e-15, -5.7692698773836694e-15, -2.34931495328177e-14, -2.4121618632559298e-14, -1.4240043282421998e-14, 8.925268126498792e-15, -1.9422743019961197e-15, 7.308034929314451e-14, -1.21463465817379e-14, 3.54410415515754e-14, -1.1357269065187199e-13, 1.1141214367970302e-13, -1.02593597126525e-13, 9.02717238412258e-14, -1.14206053450205e-13, 1.40351467505759e-13, -1.49336398713246e-13, -1.2705576317848e-13, 7.60365082436195e-14, -6.24086499966383e-14, -6.71246127076475e-14, -7.085441162653841e-15, 7.73914110829131e-16, -1.38271562715623e-14, -2.38653976927459e-14, 2.9438128411643203e-14, -7.22810967133925e-14, -6.523431227994709e-14, 1.30355868417077e-13, 3.9343710148618e-14, -4.81677951427965e-14, -4.16741705901112e-15, 4.97791637464443e-14, -4.1720176922874796e-14, -5.556243002243241e-14, 2.8436071015560506e-14, 1.2038728044337298e-14]

In [25]:
x == y

True

In [30]:
label.columns[:5]

Index(['AFIB', 'Abnormal.Gait', 'Abnormal.LFTs', 'Aching.joints', 'Acidosis'], dtype='object')